## Preparação

In [1]:
# %%capture
# !pip install google-colab-shell
# # !apt-get install -y p7zip-full
# !pip install mlflow
# !pip install -U kaleido

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23

In [ ]:
# from google_colab_shell import getshell
# getshell()

In [4]:
# !ls

In [5]:
# !mv -f path .\mlruns\4\d70fc136e3fa41c6b8a588000befb64c\params\

In [6]:
# %cd /content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23/mlruns/
# # !7z x /content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23/mlruns/4.7z
# !unzip /content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23/mlruns/4.zip
# %cd /content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23/

## Experimento

In [7]:
import os
import json
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd
import yaml
from scipy import stats
from scipy.stats import wilcoxon
from scipy.stats import mannwhitneyu
from scipy.stats import iqr
import statistics
import plotly.io as pio


In [8]:
class Metric(object):
  def __init__(self, data, model_path, problem, alpha=0.05, name=None):
    self.data = data
    self.name = name
    self.model_path = model_path
    self.problem = problem
    if len(data) > 1:
      self.avg = statistics.mean(data)
      self.median = statistics.median(data)
      self.std = statistics.stdev(data)
      self.iqr = iqr(data)
      self.var = statistics.variance(data)
      self.alpha = alpha
      self.stat, self.p = stats.shapiro(data)
      if self.p > self.alpha:
        self.gaussian = True
      else:
        self.gaussian = False

  def _std(self):
    squared_deviations = [(x - self.avg)**2 for x in self.data]
    return (sum(squared_deviations) / len(self.sample))**0.5, squared_deviations

  def __repr__(self):
    # gauss_str = lambda x: 'looks' if  x is True else 'does NOT look'
    return f'[model path: {self.model_path}, problem: {self.problem}, metric name: {self.name}, len: {len(self.data)}, gauss: {self.gaussian}, avg: {self.avg}, median: {self.median}, std: {self.std}, iqr: {self.iqr},]'

In [9]:
import pandas as pd
import statistics
from scipy import stats
from scipy.stats import iqr

class Metric(object):
    def __init__(self, data, model_path, problem, alpha=0.05, name=None):
        self.data = data
        self.name = name
        self.model_path = model_path
        self.problem = problem
        if len(data) > 1:
            self.avg = statistics.mean(data)
            self.median = statistics.median(data)
            self.std = statistics.stdev(data)
            self.iqr = iqr(data)
            self.var = statistics.variance(data)
            self.alpha = alpha
            self.stat, self.p = stats.shapiro(data)
            if self.p > self.alpha:
                self.gaussian = True
            else:
                self.gaussian = False

    def _std(self):
        squared_deviations = [(x - self.avg)**2 for x in self.data]
        return (sum(squared_deviations) / len(self.data))**0.5, squared_deviations

    def __repr__(self):
        return f'[model path: {self.model_path}, problem: {self.problem}, metric name: {self.name}, len: {len(self.data)}, gauss: {self.gaussian}, avg: {self.avg}, median: {self.median}, std: {self.std}, iqr: {self.iqr},]'

    @classmethod
    def to_dataframe(cls, metrics):
        data = [{'model path': metric.model_path, 'problem': metric.problem, 'median': metric.median} for metric in metrics]
        return pd.DataFrame(data)

# Example usage:
# metrics = [Metric(data=[1, 2, 3], model_path='path1', problem='problem1'),
#            Metric(data=[4, 5, 6], model_path='path2', problem='problem2')]
# df = Metric.to_dataframe(metrics)
# print(df)


In [ ]:
class StatTest(object):
  def __init__(self, test_name, sample1, sample2, name1=None, name2=None, alpha=0.05):
    self.test_name = test_name
    if test_name.lower() == 'wilcoxon':
      self.test = wilcoxon
    if test_name.lower() == 'ttest_rel':
      self.test = stats.ttest_rel
    if test_name.lower() == 'ttest_ind':
      self.test = stats.ttest_ind
    if test_name.lower() == 'mannwhitneyu':
      self.test = mannwhitneyu

    self.sample1 = sample1
    self.sample2 = sample2
    self.name1 = name1
    self.name2 = name2
    self.alpha = alpha
    self.stat, self.p, self.result = self._test()
    self.eff_size = self._effect_size(self.stat)

  def _test(self):
    stat, p = self.test(self.sample1,self.sample2)
    if p > self.alpha:
        return stat, p, True
    else:
        return stat, p, False
    
  # Calculate the rank-biserial correlation as effect size
  def _effect_size(self, u_stat, n = 100, m = 100):
    return 1 - (2 * u_stat) / (n * m)
     

  def __repr__(self) -> str:
    if self.result is True:
      return f'[{self.test_name}: {self.name1} VS {self.name2}, eff_size={self.eff_size}, stat={self.stat}, p={self.p} ,alpha={self.alpha}, Result: There is NO significant difference between the two models.'
    else:
      return f'[{self.test_name}: {self.name1} VS {self.name2}, eff_size={self.eff_size}, stat={self.stat}, p={self.p} ,alpha={self.alpha}, Result: There is a significant difference between the two models.'

In [ ]:
import numpy as np
import pdb

class Experiment():
  def __init__(self,mlruns_path,experiment):
    self.mlruns_path = mlruns_path
    self.experiment_path = os.path.join(mlruns_path, str(experiment))
    self.experiment_id = experiment
    self.runs = self.list_runs()
    # Convert list to float in 'cost_best_avg'
    self.runs['cost_best_avg'] = self.runs['cost_best_avg'].apply(lambda x: x[0] if isinstance(x, list) else x)

    # Replace 0 with np.nan
    self.runs['cost_best_avg'].replace(0, np.nan, inplace=True)
    self.runs.dropna(inplace=True)

    # Group by 'model_path' and 'problem', and keep only the row with the minimum 'cost_best_avg'
    self.runs = self.runs.loc[self.runs.groupby(['model_path', 'problem'])['cost_best_avg'].idxmin()]



  def list_runs(self):
    experiments_runs = []
    metrics = ['cost_best_avg', 'cost_best_std', 'steps_to_converge_avg', 'steps_to_converge_std']
    for run in os.listdir(self.experiment_path):
      run_path = os.path.join(self.experiment_path, run)
      if os.path.isdir(run_path) and run != "artifacts":
        params = self.read_params(run_path)
        path = params.get('path', 'N/A')
        problem = params.get('problem', 'N/A')

        run_metrics = {}
        for metric in metrics:
          run_metrics[metric] = self.read_metric(run_path, metric).data

        experiments_runs.append({
                        'experiment_id': self.experiment_id,
                        'experiment_name': self.get_experiment_name(self.mlruns_path, str(self.experiment_id)),
                        'run_id': run,
                        'run_path':run_path,
                        'model_path': path,
                        'problem': problem,
                        # 'cost_best_avg': run_metrics['cost_best_avg']['values'] if run_metrics['cost_best_avg'] != 'N/A' else 'N/A',
                        # 'cost_best_std': run_metrics['cost_best_std']['values'] if run_metrics['cost_best_std'] != 'N/A' else 'N/A',
                        # 'steps_to_converge_avg': run_metrics['steps_to_converge_avg']['values'] if run_metrics['steps_to_converge_avg'] != 'N/A' else 'N/A',
                        # 'steps_to_converge_std': run_metrics['steps_to_converge_std']['values'] if run_metrics['steps_to_converge_std'] != 'N/A' else 'N/A',

                        'cost_best_avg': run_metrics['cost_best_avg'] if run_metrics['cost_best_avg'] != 'N/A' else 'N/A',
                        'cost_best_std': run_metrics['cost_best_std'] if run_metrics['cost_best_std'] != 'N/A' else 'N/A',
                        'steps_to_converge_avg': run_metrics['steps_to_converge_avg'] if run_metrics['steps_to_converge_avg'] != 'N/A' else 'N/A',
                        'steps_to_converge_std': run_metrics['steps_to_converge_std'] if run_metrics['steps_to_converge_std'] != 'N/A' else 'N/A'
                    })
    return pd.DataFrame(experiments_runs)

  def read_params(self, run_folder):
    params = {}
    params_path = os.path.join(run_folder, 'params')
    if os.path.exists(params_path):
        for param_file in os.listdir(params_path):
            with open(os.path.join(params_path, param_file), 'r') as f:
                params[param_file] = f.read().strip()
    return params

  def get_experiment_name(self, mlruns_path, experiment_id):
    metadata_path = os.path.join(mlruns_path, experiment_id, "meta.yaml")
    experiment_name = None

    if os.path.exists(metadata_path):
        with open(metadata_path, 'r') as f:
            yaml_data = yaml.safe_load(f)
            experiment_name = yaml_data.get('name', None)

    return experiment_name

  def read_metric(self, run_folder, metric_name, steps_per_epoch=None):
      metric_file = os.path.join(run_folder, 'metrics', metric_name)
      params = self.read_params(run_folder)
      if os.path.exists(metric_file):
          with open(metric_file, 'r') as f:
              lines = f.readlines()
              steps = []
              values = []
              for line in lines:
                  tokens = line.strip().split()
                  if len(tokens) == 3:
                      _, value, step = tokens
                      # steps.append(int(step))
                      values.append(float(value))
              try:
                metric_obj = Metric(values, params['path'], params['problem'], name = metric_name)
              except:
                metric_obj = Metric([0],'','')
              return metric_obj
              # return {'name': metric_name, 'values': values, 'run_folder':run_folder}
      return Metric([0],'','')


  def __str__(self):
    return str(self.runs)

  def __repr__(self):
    return str(self.runs)

  def get_runs_list(self):
    return self.runs

  def get_run_by_param(self, **kwargs):
    run_filtered = self.runs
    for key, value in kwargs.items():
      run_filtered = run_filtered[run_filtered[key] == value]
    return run_filtered

  def plot_line_metric(self,**metric):
    data = pd.DataFrame(metric)

    # Create the line plot using plotly.graph_objs
    fig = go.Figure()

    for column in data.columns:
        column_parts = column.split('-')
        column_name = column_parts[0]
        metric_name = column_parts[1]
        fig.add_trace(go.Scatter(y=data[column], mode='lines', name=column_name))

    fig.update_layout(title='Metric: '+ metric_name, #'Cost Evolution during One Epoch'
                      xaxis_title='Steps',
                      yaxis_title='Cost',
                      width=800, height=600)

    fig.show()

  def boxplot_metric(self,**metric):
    data = pd.DataFrame(metric)

    # Create the boxplot using plotly.graph_objs
    fig = make_subplots(rows=1, cols=1)

    for column in data.columns:
        column_parts = column.split('-')
        column_name = column_parts[0]
        metric_name = column_parts[1]
        fig.add_trace(go.Box(y=data[column], name=column_name, boxpoints='outliers', showlegend=True))

    fig.update_layout(title='Metric: '+ metric_name, width=800, height=600)
    fig.show()


In [ ]:
# Set the path to your 'mlruns' folder on Google Drive
mlruns_path = './mlruns/'

In [ ]:
exp = Experiment(mlruns_path,4)

In [ ]:
exp.runs

In [ ]:
# import pickle

# # Pickling the object
# with open('/content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23/mlruns/exp.pkl', 'wb') as f:
#     pickle.dump(exp, f)


In [ ]:
# import pickle

# # Unpickling the object
# with open('/content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23/mlruns/exp.pkl', 'rb') as f:
#     exp = pickle.load(f)

# print(exp)

In [ ]:
exp.runs.to_csv('./figures/tabela_resultados.csv', index=False, sep=';')

In [ ]:
# name_dict = {
#     './square_cos_5_7p_only_im_finetuned_V2_exp1': 'L20_Swarm_IL_r',
#     './square_cos_5_7p_only_im_finetuned_V2_exp2': 'L20_Swarm_IL_s',
#     './square_cos_5_7p_baseline_exp2': 'L20_Swarm_r',
#     './square_cos_5_7p_baseline_exp3': 'L20_Swarm_s',
#     '.\square_cos_5_Adam_exp1': 'Adam',
#     '.\quadratic_5_Adam_exp1': 'Adam',
#     '.\quadratic_5_PSO': 'PSO',
#     '.\square_cos_2_PSO': 'PSO',
#     '.\square_cos_5_PSO': 'PSO',
#     '.\quadratic_2_PSO': 'PSO',
#     './quadratic_2_7p_baseline_exp1': 'L20_Swarm_s',
#     './quadratic_2_7p_only_im_finetuned_V2_exp1': 'L20_Swarm_IL_s',
#     './square_cos_2_7p_only_im_finetuned_V2_exp2': 'L20_Swarm_IL_r',
#     './square_cos_2_7p_baseline_exp1': 'L2O_Swarm_r',
#     './quadratic_5_7p_baseline_exp3': 'L2O_Swarm_s',
#     './quadratic_2_7p_baseline_exp3': 'L2O_Swarm_s',
#     '.\square_cos_5_Adam_exp1': 'Adam',
#     './quadratic_5_7p_only_im_finetuned_V2_exp1': 'L2O_Swarm_IL_s',
#     '.\quadratic_2_Adam_exp1': 'Adam',
#     '.\square_cos_2_Adam_exp1': 'Adam',
#     './quadratic_2_7p_baseline_exp2': 'L2O_Swarm_s',
#     './quadratic_2_7p_baseline_exp3': 'L2O_Swarm_s',
#     './quadratic_2_7p_baseline_exp4': 'L2O_Swarm_s',
#     './quadratic_2_7p_only_im_finetuned_V2_exp2': 'L20_Swarm_IL_s',
#     './quadratic_2_7p_only_im_finetuned_V2_exp3': 'L20_Swarm_IL_s',
#     './quadratic_5_7p_baseline_exp1': 'L2O_Swarm_s',
#     './quadratic_5_7p_baseline_exp2': 'L2O_Swarm_s',
#     './quadratic_5_7p_baseline_exp3': 'L2O_Swarm_s',
#     './rosenbrock_5_7p_baseline_exp1': 'L2O_Swarm_rb',
#     './rosenbrock_5_7p_baseline_exp2': 'L2O_Swarm_rb',
#     './rosenbrock_5_7p_baseline_exp3': 'L2O_Swarm_rb',
#     './rosenbrock_2_7p_baseline_exp1': 'L2O_Swarm_rb',
#     './rosenbrock_2_7p_baseline_exp2': 'L2O_Swarm_rb',
#     './rosenbrock_2_7p_baseline_exp3': 'L2O_Swarm_rb',
#     './rosenbrock_5_7p_only_im_finetuned_V2_exp1': 'L2O_Swarm_IL_rb',
#     './rosenbrock_5_7p_only_im_finetuned_V2_exp2': 'L2O_Swarm_IL_rb',
#     './rosenbrock_5_7p_only_im_finetuned_V2_exp3': 'L2O_Swarm_IL_rb',
#     './rosenbrock_2_7p_only_im_finetuned_V2_exp1': 'L2O_Swarm_IL_rb',
#     './rosenbrock_2_7p_only_im_finetuned_V2_exp2': 'L2O_Swarm_IL_rb',
#     './rosenbrock_2_7p_only_im_finetuned_V2_exp3': 'L2O_Swarm_IL_rb',
#     '.\\rosenbrock_2_Adam_exp1': 'Adam',
#     '.\\rosenbrock_2_Adam_exp2': 'Adam',
#     '.\\rosenbrock_2_Adam_exp3': 'Adam',
#     '.\\rosenbrock_5_Adam_exp1': 'Adam',
#     '.\\rosenbrock_5_Adam_exp2': 'Adam',
#     '.\\rosenbrock_5_Adam_exp3': 'Adam',
#     '.\PSO': 'PSO',
#     './rosenbrock_5_7p_sumfx_im_V1_exp1': 'L2O_Swarm_IL1_rb',
#     './quadratic_2_7p_sumfx_im_V1_exp1': 'L2O_Swarm_IL1_s',
#     './quadratic_5_7p_sumfx_im_V1_exp1': 'L2O_Swarm_IL1_s',
#     './weierstrass_5_7p_only_im_finetuned_V2_exp1_fixed': 'L2O_Swarm_IL_w',
#     './weierstrass_5_7p_baseline_exp1': 'L2O_Swarm_w',
#     './weierstrass_2_7p_baseline_exp1': 'L2O_Swarm_w',
#     './weierstrass_2_7p_baseline_exp2': 'L2O_Swarm_w',
#     './weierstrass_2_7p_baseline_exp3': 'L2O_Swarm_w',
#     './weierstrass_2_7p_only_im_finetuned_V2_exp1': 'L2O_Swarm_IL_w',
#     './weierstrass_2_7p_only_im_finetuned_V2_exp2': 'L2O_Swarm_IL_w',
#     './weierstrass_2_7p_only_im_finetuned_V2_exp3': 'L2O_Swarm_IL_w',
#     './square_cos_5_7p_only_RL_V2_exp3': 'only RL',
#     './ackley_5_7p_baseline_exp1': 'L2O_Swarm_a',
#     './ackley_5_7p_baseline_exp2': 'L2O_Swarm_a',
#     './ackley_5_7p_baseline_exp3': 'L2O_Swarm_a',
#     './ackley_5_7p_only_im_finetuned_V2_exp1': 'L2O_Swarm_IL_a',
#     './ackley_5_7p_only_im_finetuned_V2_exp2': 'L2O_Swarm_IL_a',
#     './ackley_5_7p_only_im_finetuned_V2_exp3': 'L2O_Swarm_IL_a',
#     './ackley_2_7p_baseline_exp1': 'L2O_Swarm_a',
#     './ackley_2_7p_baseline_exp2': 'L2O_Swarm_a',
#     './ackley_2_7p_baseline_exp3': 'L2O_Swarm_a',
#     './ackley_2_7p_only_im_finetuned_V2_exp1': 'L2O_Swarm_IL_a',
#     './ackley_2_7p_only_im_finetuned_V2_exp2': 'L2O_Swarm_IL_a',
#     './ackley_2_7p_only_im_finetuned_V2_exp3': 'L2O_Swarm_IL_a',
# }

In [ ]:
name_dict = {
    './square_cos_5_7p_only_im_finetuned_V2_exp1': 'proposto_r',
    './square_cos_5_7p_only_im_finetuned_V2_exp2': 'proposto_s',
    './square_cos_5_7p_baseline_exp2': 'referencia_r',
    './square_cos_5_7p_baseline_exp3': 'referencia_s',
    '.\square_cos_5_Adam_exp1': 'Adam',
    '.\quadratic_5_Adam_exp1': 'Adam',
    '.\quadratic_5_PSO': 'PSO',
    '.\square_cos_2_PSO': 'PSO',
    '.\square_cos_5_PSO': 'PSO',
    '.\quadratic_2_PSO': 'PSO',
    './quadratic_2_7p_baseline_exp1': 'referencia_s',
    './quadratic_2_7p_only_im_finetuned_V2_exp1': 'proposto_s',
    './square_cos_2_7p_only_im_finetuned_V2_exp2': 'proposto_r',
    './square_cos_2_7p_baseline_exp1': 'referencia_r',
    './quadratic_5_7p_baseline_exp3': 'referencia_s',
    './quadratic_2_7p_baseline_exp3': 'referencia_s',
    '.\square_cos_5_Adam_exp1': 'Adam',
    './quadratic_5_7p_only_im_finetuned_V2_exp1': 'proposto_s',
    '.\quadratic_2_Adam_exp1': 'Adam',
    '.\square_cos_2_Adam_exp1': 'Adam',
    './quadratic_2_7p_baseline_exp2': 'referencia_s',
    './quadratic_2_7p_baseline_exp3': 'referencia_s',
    './quadratic_2_7p_baseline_exp4': 'referencia_s',
    './quadratic_2_7p_baseline_exp5': 'referencia_s',
    './quadratic_2_7p_only_im_finetuned_V2_exp2': 'proposto_s',
    './quadratic_2_7p_only_im_finetuned_V2_exp3': 'proposto_s',
    './quadratic_5_7p_baseline_exp1': 'referencia_s',
    './quadratic_5_7p_baseline_exp2': 'referencia_s',
    './quadratic_5_7p_baseline_exp3': 'referencia_s',
    './rosenbrock_5_7p_baseline_exp1': 'referencia_rb',
    './rosenbrock_5_7p_baseline_exp2': 'referencia_rb',
    './rosenbrock_5_7p_baseline_exp3': 'referencia_rb',
    './rosenbrock_2_7p_baseline_exp1': 'referencia_rb',
    './rosenbrock_2_7p_baseline_exp2': 'referencia_rb',
    './rosenbrock_2_7p_baseline_exp3': 'referencia_rb',
    './rosenbrock_5_7p_only_im_finetuned_V2_exp4': 'proposto_rb',
    './rosenbrock_5_7p_only_im_finetuned_V2_exp1': 'proposto_rb',
    './rosenbrock_5_7p_only_im_finetuned_V2_exp2': 'proposto_rb',
    './rosenbrock_5_7p_only_im_finetuned_V2_exp3': 'proposto_rb',
    './rosenbrock_2_7p_only_im_finetuned_V2_exp1': 'proposto_rb',
    './rosenbrock_2_7p_only_im_finetuned_V2_exp2': 'proposto_rb',
    './rosenbrock_2_7p_only_im_finetuned_V2_exp3': 'proposto_rb',
    '.\\rosenbrock_2_Adam_exp1': 'Adam',
    '.\\rosenbrock_2_Adam_exp2': 'Adam',
    '.\\rosenbrock_2_Adam_exp3': 'Adam',
    '.\\rosenbrock_5_Adam_exp1': 'Adam',
    '.\\rosenbrock_5_Adam_exp2': 'Adam',
    '.\\rosenbrock_5_Adam_exp3': 'Adam',
    '.\PSO': 'PSO',
    './rosenbrock_5_7p_sumfx_im_V1_exp1': 'proposto1_rb',
    './quadratic_2_7p_sumfx_im_V1_exp1': 'proposto1_s',
    './quadratic_5_7p_sumfx_im_V1_exp1': 'proposto1_s',
    './weierstrass_5_7p_only_im_finetuned_V2_exp1_fixed': 'proposto_w',
    './weierstrass_5_7p_only_im_finetuned_V2_exp1': 'proposto_w',
    './weierstrass_5_7p_only_im_finetuned_V2_exp2': 'proposto_w',
    './weierstrass_5_7p_only_im_finetuned_V2_exp3': 'proposto_w',
    './weierstrass_5_7p_baseline_exp1': 'referencia_w',
    './weierstrass_2_7p_baseline_exp1': 'referencia_w',
    './weierstrass_2_7p_baseline_exp2': 'referencia_w',
    './weierstrass_2_7p_baseline_exp3': 'referencia_w',
    './weierstrass_2_7p_only_im_finetuned_V2_exp1': 'proposto_w',
    './weierstrass_2_7p_only_im_finetuned_V2_exp2': 'proposto_w',
    './weierstrass_2_7p_only_im_finetuned_V2_exp3': 'proposto_w',
    './weierstrass_2_7p_only_im_finetuned_V2_exp4': 'proposto_w',
    '.\weierstrass_5_Adam_exp1': 'Adam',
    '.\weierstrass_2_Adam_exp1': 'Adam',
    './ackley_5_7p_only_im_finetuned_V2_exp1': 'proposto_w',
    './weierstrass_smoothed_5_7p_baseline_exp1': 'proposto_w',
    './square_cos_5_7p_only_RL_V2_exp3': 'only RL',
    './ackley_5_7p_baseline_exp1': 'referencia_a',
    './ackley_5_7p_baseline_exp2': 'referencia_a',
    './ackley_5_7p_baseline_exp3': 'referencia_a',
    './ackley_5_7p_only_im_finetuned_V2_exp1': 'proposto_a',
    './ackley_5_7p_only_im_finetuned_V2_exp2': 'proposto_a',
    './ackley_5_7p_only_im_finetuned_V2_exp3': 'proposto_a',
    './ackley_2_7p_baseline_exp1': 'referencia_a',
    './ackley_2_7p_baseline_exp2': 'referencia_a',
    './ackley_2_7p_baseline_exp3': 'referencia_a',
    './ackley_2_7p_only_im_finetuned_V2_exp1': 'proposto_a',
    './ackley_2_7p_only_im_finetuned_V2_exp2': 'proposto_a',
    './ackley_2_7p_only_im_finetuned_V2_exp3': 'proposto_a',
    './square_cos_5_7p_only_sumfx_exp1': 'only_RL_r',
}

## Quadratic 5d

In [ ]:
from math import log10

problem = 'quadratic'

path_problem = []

path_problem.append(('./quadratic_5_7p_baseline_exp1',problem))
path_problem.append(('./quadratic_5_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./square_cos_5_7p_baseline_exp2',problem))
path_problem.append(('./square_cos_5_7p_only_im_finetuned_V2_exp1',problem))
path_problem.append(('./square_cos_5_7p_only_sumfx_exp1',problem))

path_problem.append(('./rosenbrock_5_7p_baseline_exp1',problem))
path_problem.append(('./rosenbrock_5_7p_only_im_finetuned_V2_exp4',problem))

path_problem.append(('./weierstrass_5_7p_baseline_exp1',problem))
path_problem.append(('./weierstrass_5_7p_only_im_finetuned_V2_exp1_fixed',problem))

path_problem.append(('./ackley_5_7p_baseline_exp2',problem))
path_problem.append(('./ackley_5_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('.\quadratic_5_Adam_exp1',problem))
path_problem.append(('.\quadratic_5_PSO',problem))


plot_dict = {}
# metric_name = 'epoch_best_cost'
# metric_name = 'epoch_best_cost'
run_metric_list = []
colors = [
'rgb(0, 114, 178)', # Strong Blue
'rgb(230, 159, 0)', # Orange
'rgb(0, 158, 115)', # Blue Green
'rgb(86, 180, 233)', # Sky Blue
'rgb(213, 94, 0)', # Dark Orange
'rgb(204, 121, 167)', # Pale Violet Red
'rgb(214, 39, 40)', # Red
'rgb(148, 103, 189)', # Purple
'rgb(128, 128, 128)', # Grey
# 'rgb(240, 228, 66)', # Yellow
'rgb(255, 87, 51)', # Orange Red
'rgb(0, 191, 165)', # Turquoise
'rgb(75, 101, 132)', # Slate Blue
'rgb(153, 102, 204)', # Medium Purple
'rgb(255, 235, 59)', # Bright Yellow
'rgb(152, 251, 152)', # Pale Green
'rgb(66, 134, 244)', # Dodger Blue
'rgb(0, 179, 134)', # Medium Turquoise
'rgb(255, 165, 0)', # Orange
'rgb(153, 50, 204)' # Dark Orchid
]
num_epochs = 100
num_steps = 250
for idx, (model_path, problem) in enumerate(path_problem):
    run_dict = {'model_path':model_path,'problem':problem}
    #   if run_dict['model_path']=='.\square_cos_5_Adam_exp1':
    #       run_dict['problem']='rastrigin'
    #   if run_dict['model_path']=='.\square_cos_5_PSO':
    #       run_dict['problem']='quadratic'
    run = exp.get_run_by_param(**run_dict)

    if run.empty is False:
        run_path = run['run_path'].item()
#         run_metric = exp.read_metric(run_path, metric_name)
        ##################################
        # Fetch the data for each epoch
        data_list = []
        for epoch in range(1, num_epochs + 1):
            metric_name = f"epoch_{epoch}_cost_min"
            run_metric = exp.read_metric(run_path, metric_name)
            # print(run_metric.name)
            epoch_data = pd.DataFrame({
                'epoch': epoch,
                'step': np.arange(1, num_steps + 1),
                'model_path': run_metric.model_path,
                'value': run_metric.data  # assuming this is a list or array of length num_steps
            })
            data_list.append(epoch_data)

        # Concatenate all dataframes
        data = pd.concat(data_list)

        # calculate mean values for line plot
        min_values = data.groupby('epoch')['value'].min() # (self, data, model_path, problem, alpha=0.05, name=None):
        
        ##############
        run_metric_list.append(Metric(min_values, run_metric.model_path, problem , name=metric_name))
#         plot_dict[run_metric.model_path+ '-' +run_metric.name] = Metric(min_values.values, run_metric.model_path, problem , name=metric_name)
        plot_dict[run_metric.model_path+ '-' +run_metric.name] = min_values
    else:
        pdb.set_trace()
        print('Not found: ', run_dict)

data = pd.DataFrame(plot_dict)

# Create the boxplot using plotly.graph_objs
fig = make_subplots(rows=1, cols=1)

for idx, column in enumerate(data.columns):
    column_parts = column.split('-')
    column_name = column_parts[0]
    metric_name = column_parts[1]
    fig.add_trace(go.Box(y=data[column], name=name_dict[column_name], boxpoints='outliers', showlegend=True,
                         marker=dict(color=colors[idx], 
                                     line=dict(width=0.3),
#                                      outliercolor='rgba(219, 64, 82, 0.6)', # Optional: set color for outlier points
#                                      outliersize=6,
                                    )))

fig.update_layout(#title='Metric: '+ metric_name,
                  plot_bgcolor='rgb(255, 255, 255)',
                  width=800,
                  height=600,
                  xaxis=dict(
#                       title_text='Evaluation iterations',
                      gridcolor='rgb(200, 200, 200)',
                      zerolinecolor='rgb(200, 200, 200)'
                  ),
                    yaxis=dict(
                        title_text='Log10(Valor do Fitness)',
                        type="log",
                        gridcolor='rgb(200, 200, 200)',
                        zerolinecolor='rgb(200, 200, 200)',
#                         range=[log10(0.1), log10(400)]  # Adicione essa linha para focar em um subconjunto específico dos dados
                    ),
                  font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="black"
                  ),
                  showlegend=False,
                  legend=dict(
                      x=0,
                      y=1.005,
                      xanchor='left',
                      yanchor='bottom',
                      traceorder="normal",
                      font=dict(
                          family="sans-serif",
                          size=12,
                          color="black"
                      ),
                      bgcolor='rgba(229, 236, 246, 0)',
                      bordercolor="Black",
                      borderwidth=1
                  ))
fig.show()


In [ ]:


pio.write_image(fig, './figures/quadratic_5d_boxplot.pdf', scale=10) #width=800, height=600, scale=2

### Quadratic 5d - Statistic Tests 

In [ ]:
def pairwise_test(samples):
    n = len(samples)
    for i in range(n):
        for j in range(i+1, n):
            # print(f'Testing pair ({samples[i]}, {samples[j]})')
            test = StatTest('mannwhitneyu', samples[i].data, samples[j].data, name1 = samples[i].model_path, name2 = samples[j].model_path)
#             if test.result is True:
#               print(samples[i])
#               print(samples[j])
            print(test)
#             print('\n')
            # replace this print statement with your actual test function
            # for example: test_result = test(samples[i], samples[j])
            # where "test" is your test function

# samples = ['A', 'B', 'C', 'D', 'E']
pairwise_test(run_metric_list)




In [ ]:
df = Metric.to_dataframe(run_metric_list)
df

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

problem = 'quadratic'

run_path_list = []
for (model_path, problem) in path_problem:
  run_dict = {'model_path': model_path,'problem': problem}
  # gambiarra pra plotar Adam
  # if (run_dict['model_path']=='.\square_cos_5_Adam_exp1'): # and (run_dict['problem']=='square_cos_5'):
  #     run_dict['problem']='rastrigin'
  run = exp.get_run_by_param(**run_dict)
  if run.empty is False:
    run_path_list.append(run['run_path'].item())

# Assume you have the `exp` object available and the number of epochs
num_epochs = 100
num_steps = 250


# Initialize the figure
fig = go.Figure()

# fetch and plot data for each optimizer
for opt_idx, run_path in enumerate(run_path_list):

    data_list = []

    # Fetch the data for each epoch
    for epoch in range(1, num_epochs + 1):
        metric_name = f"epoch_{epoch}_cost_min"
        run_metric = exp.read_metric(run_path, metric_name)
        # print(run_metric.name)
        epoch_data = pd.DataFrame({
            'epoch': epoch,
            'step': np.arange(1, num_steps + 1),
            'model_path': run_metric.model_path,
            'value': run_metric.data  # assuming this is a list or array of length num_steps
        })
        data_list.append(epoch_data)

    # Concatenate all dataframes
    data = pd.concat(data_list)
    # calculate mean values for line plot
#     mean_values = data.groupby('epoch')['value'].mean()
    # Agrupe por 'step' e calcule o mínimo para cada grupo
    mean_values = data.groupby('step')['value'].mean()
#     print(mean_values)

#     # create box plots for each 10th step
#     for step in range(1, num_steps+1):
#         if step % 5 == 0:  # only create boxplot for every 10th step
#             fig.add_trace(go.Box(
#                 y=data.loc[data['step'] == step, 'value'],
#                 x=[step] * len(data.loc[data['step'] == step, 'value']),  # specify x-position for each point in the box plot
#                 name=str(step),
#                 showlegend=False,
#                 boxpoints='outliers',#'outliers',
#                 width=0.3,  # Adjust this value to make the box narrower or wider
#                 marker=dict(
#                   size=0.5,
#                   color=colors[opt_idx][:3] + 'a'+ colors[opt_idx][3:-1]+',0.1)',  # use different color for each optimizer
#                   line=dict(width=0.3),  # change width of the box outline
#                 ),
#                 line=dict(width=0.3),  # change width of the whisker lines
#             ))

    # create line plot for the mean values
    fig.add_trace(go.Scatter(
        x=mean_values.index,  # numerical x-values
        y=mean_values.values,
        mode='lines',
        name=name_dict[run_metric.model_path],
        line=dict(
          color=colors[opt_idx],
          width=3,  # adjust this for line thickness
        ),
    ))


fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        type="log",
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=800,
    height=600,
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(255, 255, 255, 0.5)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=1
    )
)

fig.update_xaxes(
    title_text='Iterações',
    tickangle=0,
    tickvals=[i for i in range(0,261,50)],
    # range=[0, 150],
    )
fig.update_yaxes(
    title_text='log10(Valor do Fitness)',
#     range=[log10(0.05), log10(22)],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/quadratic_5_line_main.pdf', scale=10) #width=800, height=600, scale=2


In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/quadratic_5_line_zoom1.pdf', scale=10) #width=800, height=600, scale=2


In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=350,
    height=500,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,1)],
    range=[246, 250],
    )
fig.update_yaxes(
    title_text='Loss',
    range=[0.125, 0.42],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/quadratic_5_line_zoom2.pdf', scale=10) #width=800, height=600, scale=2

## rastrigin 5d

In [ ]:
import pdb
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

problem = 'square_cos_5'
path_problem = []

path_problem.append(('./quadratic_5_7p_baseline_exp3',problem))
path_problem.append(('./quadratic_5_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./square_cos_5_7p_baseline_exp2',problem))
path_problem.append(('./square_cos_5_7p_only_im_finetuned_V2_exp1',problem))
# path_problem.append(('./square_cos_5_7p_only_sumfx_exp1',problem))

path_problem.append(('./rosenbrock_5_7p_baseline_exp3',problem))
path_problem.append(('./rosenbrock_5_7p_only_im_finetuned_V2_exp2',problem))

path_problem.append(('./weierstrass_5_7p_baseline_exp1',problem))
path_problem.append(('./weierstrass_5_7p_only_im_finetuned_V2_exp1_fixed',problem))

path_problem.append(('./ackley_5_7p_baseline_exp1',problem))
path_problem.append(('./ackley_5_7p_only_im_finetuned_V2_exp2',problem))


# path_problem.append(('./rosenbrock_5_7p_baseline_exp3',problem))

# path_problem.append(('./square_cos_5_7p_only_RL_V2_exp3',problem))

# path_problem.append(('./square_cos_5_7p_baseline_exp3',problem))
path_problem.append(('.\square_cos_5_Adam_exp1',problem))
path_problem.append(('.\square_cos_5_PSO',problem)) #.\square_cos_5_PSO

plot_dict = {}
metric_name = 'epoch_best_cost'
run_metric_list = []
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]


num_epochs = 100
num_steps = 250
for idx, (model_path, problem) in enumerate(path_problem):
    run_dict = {'model_path':model_path,'problem':problem}
    if run_dict['model_path']=='.\square_cos_5_Adam_exp1':
        run_dict['problem']='rastrigin'
    if run_dict['model_path']=='.\square_cos_5_PSO':
        run_dict['problem']='quadratic'
    run = exp.get_run_by_param(**run_dict)

    if run.empty is False:
        run_path = run['run_path'].item()
#         run_metric = exp.read_metric(run_path, metric_name)
        ##################################
        # Fetch the data for each epoch
        data_list = []
        for epoch in range(1, num_epochs + 1):
            metric_name = f"epoch_{epoch}_cost_min"
            run_metric = exp.read_metric(run_path, metric_name)
            # print(run_metric.name)
            epoch_data = pd.DataFrame({
                'epoch': epoch,
                'step': np.arange(1, num_steps + 1),
                'model_path': run_metric.model_path,
                'value': run_metric.data  # assuming this is a list or array of length num_steps
            })
            data_list.append(epoch_data)

        # Concatenate all dataframes
        data = pd.concat(data_list)

        # calculate mean values for line plot
        min_values = data.groupby('epoch')['value'].min() # (self, data, model_path, problem, alpha=0.05, name=None):
        
        ##############
        run_metric_list.append(Metric(min_values, run_metric.model_path, problem , name=metric_name))
#         plot_dict[run_metric.model_path+ '-' +run_metric.name] = Metric(min_values.values, run_metric.model_path, problem , name=metric_name)
        plot_dict[run_metric.model_path+ '-' +run_metric.name] = min_values
    else:
        pdb.set_trace()
        print('Not found: ', run_dict)

data = pd.DataFrame(plot_dict)

# Create the boxplot using plotly.graph_objs
fig = make_subplots(rows=1, cols=1)

for idx, column in enumerate(data.columns):
    column_parts = column.split('-')
    column_name = column_parts[0]
    metric_name = column_parts[1]
    fig.add_trace(go.Box(y=data[column], name=name_dict[column_name], boxpoints='outliers', showlegend=True,
                         marker=dict(color=colors[idx], 
                                     line=dict(width=0.3),
#                                      outliercolor='rgba(219, 64, 82, 0.6)', # Optional: set color for outlier points
#                                      outliersize=6,
                                    )))

fig.update_layout(#title='Metric: '+ metric_name,
                  plot_bgcolor='rgb(255, 255, 255)',
                  width=800,
                  height=600,
                  xaxis=dict(
#                       title_text='Evaluation iterations',
                      gridcolor='rgb(200, 200, 200)',
                      zerolinecolor='rgb(200, 200, 200)'
                  ),
                    yaxis=dict(
                        title_text='Log10(Valor do Fitness)',
                        type="log",
                        gridcolor='rgb(200, 200, 200)',
                        zerolinecolor='rgb(200, 200, 200)',
#                         range=[log10(0.1), log10(400)]  # Adicione essa linha para focar em um subconjunto específico dos dados
                    ),
                  font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="black"
                  ),
                  showlegend=False,
                  legend=dict(
                      x=0,
                      y=1.005,
                      xanchor='left',
                      yanchor='bottom',
                      traceorder="normal",
                      font=dict(
                          family="sans-serif",
                          size=12,
                          color="black"
                      ),
                      bgcolor='rgba(229, 236, 246, 0)',
                      bordercolor="Black",
                      borderwidth=1
                  ))
fig.show()


In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/square_cos_5_boxplot.pdf', scale=10) #width=800, height=600, scale=2

### rastrigin 5d - Statistic Tests

In [ ]:
def pairwise_test(samples):
    n = len(samples)
    for i in range(n):
        for j in range(i+1, n):
            # print(f'Testing pair ({samples[i]}, {samples[j]})')
            test = StatTest('mannwhitneyu', samples[i].data, samples[j].data, name1 = samples[i].model_path, name2 = samples[j].model_path)
            if True: #test.result is False:
              print(samples[i])
              print(samples[j])
              print(test)
              print('\n')
            # replace this print statement with your actual test function
            # for example: test_result = test(samples[i], samples[j])
            # where "test" is your test function

# samples = ['A', 'B', 'C', 'D', 'E']
pairwise_test(run_metric_list)


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np


problem = 'square_cos_5'

run_path_list = []
for (model_path, problem) in path_problem:
  run_dict = {'model_path': model_path,'problem': problem}
  # gambiarra pra plotar Adam
  if (run_dict['model_path']=='.\square_cos_5_Adam_exp1'): # and (run_dict['problem']=='square_cos_5'):
      run_dict['problem']='rastrigin'
  if (run_dict['model_path']=='.\square_cos_5_PSO'):
      run_dict['problem']='quadratic'
  run = exp.get_run_by_param(**run_dict)
  if run.empty is False:
    run_path_list.append(run['run_path'].item())

# Assume you have the `exp` object available and the number of epochs
num_epochs = 100
num_steps = 250
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]


# colors = ['#1f77b4','#d62728','#ff7f0e', '#2ca02c','#ff7f0e','#9467bd','#8c564b','#e377c2', '#7f7f7f' ,'#bcbd22']

# Initialize the figure
fig = go.Figure()

# fetch and plot data for each optimizer
for opt_idx, run_path in enumerate(run_path_list):

    data_list = []

    # Fetch the data for each epoch
    for epoch in range(1, num_epochs + 1):
        metric_name = f"epoch_{epoch}_cost_min"
        run_metric = exp.read_metric(run_path, metric_name)
        # print(run_metric.name)
        epoch_data = pd.DataFrame({
            'epoch': epoch,
            'step': np.arange(1, num_steps + 1),
            'model_path': run_metric.model_path,
            'value': run_metric.data  # assuming this is a list or array of length num_steps
        })
        data_list.append(epoch_data)

    # Concatenate all dataframes
    data = pd.concat(data_list)
    # calculate mean values for line plot
#     mean_values = data.groupby('epoch')['value'].mean()
    # Agrupe por 'step' e calcule o mínimo para cada grupo
    mean_values = data.groupby('step')['value'].mean()
#     print(mean_values)

#     # create box plots for each 10th step
#     for step in range(1, num_steps+1):
#         if step % 5 == 0:  # only create boxplot for every 10th step
#             fig.add_trace(go.Box(
#                 y=data.loc[data['step'] == step, 'value'],
#                 x=[step] * len(data.loc[data['step'] == step, 'value']),  # specify x-position for each point in the box plot
#                 name=str(step),
#                 showlegend=False,
#                 boxpoints='outliers',#'outliers',
#                 width=0.3,  # Adjust this value to make the box narrower or wider
#                 marker=dict(
#                   size=0.5,
#                   color=colors[opt_idx][:3] + 'a'+ colors[opt_idx][3:-1]+',0.1)',  # use different color for each optimizer
#                   line=dict(width=0.3),  # change width of the box outline
#                 ),
#                 line=dict(width=0.3),  # change width of the whisker lines
#             ))

    # create line plot for the mean values
    fig.add_trace(go.Scatter(
        x=mean_values.index,  # numerical x-values
        y=mean_values.values,
        mode='lines',
        name=name_dict[run_metric.model_path],
        line=dict(
          color=colors[opt_idx],
          width=3,  # adjust this for line thickness
        ),
    ))


fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
#         type="log",
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=800,
    height=600,
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(255, 255, 255, 0.5)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=1
    )
)

fig.update_xaxes(
    title_text='Iterações',
    tickangle=0,
    tickvals=[i for i in range(0,261,50)],
    # range=[0, 150],
    )
fig.update_yaxes(
    title_text='Valor do Fitness',
#     range=[log10(0.05), log10(22)],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/square_cos_5_line_main.pdf', scale=10) #width=800, height=600, scale=2


In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=400,
    height=600,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,2)],
    range=[3, 16],
    )
fig.update_yaxes(
    title_text='Loss',
    range=[8, 15],
    )
fig.show()

In [ ]:
# import plotly.io as pio

# pio.write_image(fig, '/content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23/square_cos_5_line_zoom1.pdf', scale=10) #width=800, height=600, scale=2

# from google.colab import files

# # Assuming you have a file named 'file_name.csv' in the current directory
# files.download('/content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23/square_cos_5_line_zoom1.pdf')

In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=350,
    height=500,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,1)],
    range=[246, 250],
    )
fig.update_yaxes(
    title_text='Loss',
    range=[7, 9],
    )
fig.show()

In [ ]:
# import plotly.io as pio

# pio.write_image(fig, '/content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23/square_cos_5_line_zoom2.pdf', scale=10) #width=800, height=600, scale=2

# from google.colab import files

# # Assuming you have a file named 'file_name.csv' in the current directory
# files.download('/content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23/square_cos_5_line_zoom2.pdf')

## quadratic 2D

In [ ]:
path_problem = []
problem = 'quadratic'


# path_problem.append(('./quadratic_2_7p_only_im_finetuned_V2_exp1',problem))
# path_problem.append(('./quadratic_2_7p_sumfx_im_V1_exp1',problem))

path_problem.append(('./quadratic_2_7p_baseline_exp3',problem))
path_problem.append(('./quadratic_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./square_cos_2_7p_baseline_exp1',problem))
path_problem.append(('./square_cos_2_7p_only_im_finetuned_V2_exp2',problem))

# path_problem.append(('./quadratic_2_7p_baseline_exp4',problem))
path_problem.append(('./rosenbrock_2_7p_baseline_exp3',problem))
path_problem.append(('./rosenbrock_2_7p_only_im_finetuned_V2_exp2',problem))

path_problem.append(('./weierstrass_2_7p_baseline_exp1',problem))
path_problem.append(('./weierstrass_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./ackley_2_7p_baseline_exp1',problem))
path_problem.append(('./ackley_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('.\quadratic_2_Adam_exp1',problem))
path_problem.append(('.\quadratic_2_PSO',problem))

plot_dict = {}
# metric_name = 'epoch_best_cost'
run_metric_list = []
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]

num_epochs = 100
num_steps = 250
for idx, (model_path, problem) in enumerate(path_problem):
    run_dict = {'model_path':model_path,'problem':problem}
    #   if run_dict['model_path']=='.\square_cos_5_Adam_exp1':
    #       run_dict['problem']='rastrigin'
    #   if run_dict['model_path']=='.\square_cos_5_PSO':
    #       run_dict['problem']='quadratic'
    run = exp.get_run_by_param(**run_dict)

    if run.empty is False:
        run_path = run['run_path'].item()
#         run_metric = exp.read_metric(run_path, metric_name)
        ##################################
        # Fetch the data for each epoch
        data_list = []
        for epoch in range(1, num_epochs + 1):
            metric_name = f"epoch_{epoch}_cost_min"
            run_metric = exp.read_metric(run_path, metric_name)
            # print(run_metric.name)
            epoch_data = pd.DataFrame({
                'epoch': epoch,
                'step': np.arange(1, num_steps + 1),
                'model_path': run_metric.model_path,
                'value': run_metric.data  # assuming this is a list or array of length num_steps
            })
            data_list.append(epoch_data)

        # Concatenate all dataframes
        data = pd.concat(data_list)

        # calculate mean values for line plot
        min_values = data.groupby('epoch')['value'].min() # (self, data, model_path, problem, alpha=0.05, name=None):
        
        ##############
        run_metric_list.append(Metric(min_values, run_metric.model_path, problem , name=metric_name))
#         plot_dict[run_metric.model_path+ '-' +run_metric.name] = Metric(min_values.values, run_metric.model_path, problem , name=metric_name)
        plot_dict[run_metric.model_path+ '-' +run_metric.name] = min_values
    else:
        pdb.set_trace()
        print('Not found: ', run_dict)

data = pd.DataFrame(plot_dict)

# Create the boxplot using plotly.graph_objs
fig = make_subplots(rows=1, cols=1)

for idx, column in enumerate(data.columns):
    column_parts = column.split('-')
    column_name = column_parts[0]
    metric_name = column_parts[1]
    fig.add_trace(go.Box(y=data[column], name=name_dict[column_name], boxpoints='outliers', showlegend=True,
                         marker=dict(color=colors[idx], 
                                     line=dict(width=0.3),
#                                      outliercolor='rgba(219, 64, 82, 0.6)', # Optional: set color for outlier points
#                                      outliersize=6,
                                    )))

fig.update_layout(#title='Metric: '+ metric_name,
                  plot_bgcolor='rgb(255, 255, 255)',
                  width=800,
                  height=600,
                  xaxis=dict(
#                       title_text='Evaluation iterations',
                      gridcolor='rgb(200, 200, 200)',
                      zerolinecolor='rgb(200, 200, 200)'
                  ),
                    yaxis=dict(
                        title_text='Log10(Valor do Fitness)',
                        type="log",
                        gridcolor='rgb(200, 200, 200)',
                        zerolinecolor='rgb(200, 200, 200)',
#                         range=[log10(0.1), log10(400)]  # Adicione essa linha para focar em um subconjunto específico dos dados
                    ),
                  font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="black"
                  ),
                  showlegend=False,
                  legend=dict(
                      x=0,
                      y=1.005,
                      xanchor='left',
                      yanchor='bottom',
                      traceorder="normal",
                      font=dict(
                          family="sans-serif",
                          size=12,
                          color="black"
                      ),
                      bgcolor='rgba(229, 236, 246, 0)',
                      bordercolor="Black",
                      borderwidth=1
                  ))
fig.show()


In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/quadratic_2d_boxplot.pdf', scale=10) #width=800, height=600, scale=2



In [ ]:
pio.write_image(fig, './figures/quadratic_2d_boxplot.png', scale=10) #width=800, height=600, scale=2

In [ ]:
def pairwise_test(samples):
    n = len(samples)
    for i in range(n):
        for j in range(i+1, n):
            # print(f'Testing pair ({samples[i]}, {samples[j]})')
            test = StatTest('mannwhitneyu', samples[i].data, samples[j].data, name1 = samples[i].model_path, name2 = samples[j].model_path)
            if test.result is False:
              print(samples[i])
              print(samples[j])
              print(test)
              print('\n')
            # replace this print statement with your actual test function
            # for example: test_result = test(samples[i], samples[j])
            # where "test" is your test function

# samples = ['A', 'B', 'C', 'D', 'E']
pairwise_test(run_metric_list)


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

problem = 'quadratic'

run_path_list = []
for (model_path, problem) in path_problem:
  run_dict = {'model_path': model_path,'problem': problem}
  # gambiarra pra plotar Adam
  # if (run_dict['model_path']=='.\square_cos_5_Adam_exp1'): # and (run_dict['problem']=='square_cos_5'):
  #     run_dict['problem']='rastrigin'
  run = exp.get_run_by_param(**run_dict)
  if run.empty is False:
    run_path_list.append(run['run_path'].item())

# Assume you have the `exp` object available and the number of epochs
num_epochs = 100
num_steps = 250
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]


# colors = ['#1f77b4','#d62728','#ff7f0e', '#2ca02c','#ff7f0e','#9467bd','#8c564b','#e377c2', '#7f7f7f' ,'#bcbd22']

# Initialize the figure
fig = go.Figure()

# fetch and plot data for each optimizer
for opt_idx, run_path in enumerate(run_path_list):

    data_list = []

    # Fetch the data for each epoch
    for epoch in range(1, num_epochs + 1):
        metric_name = f"epoch_{epoch}_cost_min"
        run_metric = exp.read_metric(run_path, metric_name)
        # print(run_metric.name)
        epoch_data = pd.DataFrame({
            'epoch': epoch,
            'step': np.arange(1, num_steps + 1),
            'model_path': run_metric.model_path,
            'value': run_metric.data  # assuming this is a list or array of length num_steps
        })
        data_list.append(epoch_data)

    # Concatenate all dataframes
    data = pd.concat(data_list)
    # calculate mean values for line plot
#     mean_values = data.groupby('epoch')['value'].mean()
    # Agrupe por 'step' e calcule o mínimo para cada grupo
    mean_values = data.groupby('step')['value'].mean()
#     print(mean_values)

#     # create box plots for each 10th step
#     for step in range(1, num_steps+1):
#         if step % 5 == 0:  # only create boxplot for every 10th step
#             fig.add_trace(go.Box(
#                 y=data.loc[data['step'] == step, 'value'],
#                 x=[step] * len(data.loc[data['step'] == step, 'value']),  # specify x-position for each point in the box plot
#                 name=str(step),
#                 showlegend=False,
#                 boxpoints='outliers',#'outliers',
#                 width=0.3,  # Adjust this value to make the box narrower or wider
#                 marker=dict(
#                   size=0.5,
#                   color=colors[opt_idx][:3] + 'a'+ colors[opt_idx][3:-1]+',0.1)',  # use different color for each optimizer
#                   line=dict(width=0.3),  # change width of the box outline
#                 ),
#                 line=dict(width=0.3),  # change width of the whisker lines
#             ))

    # create line plot for the mean values
    fig.add_trace(go.Scatter(
        x=mean_values.index,  # numerical x-values
        y=mean_values.values,
        mode='lines',
        name=name_dict[run_metric.model_path],
        line=dict(
          color=colors[opt_idx],
          width=3,  # adjust this for line thickness
        ),
    ))


fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
#         type="log",
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=800,
    height=600,
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(255, 255, 255, 0.5)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=1
    )
)

fig.update_xaxes(
    title_text='Iterações',
    tickangle=0,
    tickvals=[i for i in range(0,261,50)],
    # range=[0, 150],
    )
fig.update_yaxes(
    title_text='Valor do Fitness',
#     range=[log10(0.05), log10(22)],
#     range=[0, 1],
    )
fig.show()


In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/quadratic_2_line_main.pdf', scale=10) #width=800, height=600, scale=2

# from google.colab import files

# # Assuming you have a file named 'file_name.csv' in the current directory
# files.download('/content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23/quadratic_2_line_main.pdf')



In [ ]:
import plotly.graph_objects as go

# Lendo os dados do arquivo
with open('loss_data.txt', 'r') as f:
    lines = f.readlines()

# Organizar os valores de loss por modelo
loss_by_model = [[] for _ in range(7)]

epoch_losses = []
for line in lines:
    if line.startswith("cost"):
        values = [float(v) for v in line.split('[')[1].split(']')[0].split(', ')]
        epoch_losses.append(values)
    elif "done one epoch" in line:
        for i in range(7):
            model_losses = [loss[i] for loss in epoch_losses]
            avg_loss = sum(model_losses) / len(model_losses)
            loss_by_model[i].append(avg_loss)
        epoch_losses = []

# Criar um gráfico interativo com plotly
fig = go.Figure()

epochs = list(range(1, len(loss_by_model[0]) + 1))
for i, losses in enumerate(loss_by_model):
    fig.add_trace(go.Scatter(x=epochs, y=losses, mode='lines', name=f'Model {i+1}'))

fig.update_layout(
    title="Average Loss by Epoch for Each Model",
    xaxis_title="Epoch",
    yaxis_title="Average Loss Value",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()


In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=400,
    height=500,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,10)],
    range=[0, 65],
    )
fig.update_yaxes(
    title_text='Loss',
    range=[0.2, 1],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/quadratic_5_line_zoom1.pdf', scale=10) #width=800, height=600, scale=2


In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=350,
    height=500,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,1)],
    range=[246, 250],
    )
fig.update_yaxes(
    title_text='Loss',
    range=[0.125, 0.42],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/quadratic_5_line_zoom2.pdf', scale=10) #width=800, height=600, scale=2


## rastrigin 2D

In [ ]:
path_problem = []
problem = 'square_cos_2'


# path_problem.append(('./quadratic_2_7p_sumfx_im_V1_exp1',problem))

path_problem.append(('./quadratic_2_7p_baseline_exp3',problem))
path_problem.append(('./quadratic_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./square_cos_2_7p_baseline_exp1',problem))
path_problem.append(('./square_cos_2_7p_only_im_finetuned_V2_exp2',problem))

path_problem.append(('./rosenbrock_2_7p_baseline_exp1',problem))
path_problem.append(('./rosenbrock_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./weierstrass_2_7p_baseline_exp1',problem))
path_problem.append(('./weierstrass_2_7p_only_im_finetuned_V2_exp1',problem))
# weierstrass_2_7p_only_im_300_V2_exp4

path_problem.append(('./ackley_2_7p_baseline_exp2',problem))
path_problem.append(('./ackley_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('.\square_cos_2_Adam_exp1',problem))
path_problem.append(('.\square_cos_2_PSO',problem))

plot_dict = {}
metric_name = 'epoch_best_cost'
run_metric_list = []
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]


num_epochs = 100
num_steps = 250
for idx, (model_path, problem) in enumerate(path_problem):
    run_dict = {'model_path':model_path,'problem':problem}
    if run_dict['model_path']=='.\square_cos_2_Adam_exp1':
        run_dict['problem']='rastrigin'
#     if run_dict['model_path']=='.\square_cos_5_PSO':
#         run_dict['problem']='quadratic'
    run = exp.get_run_by_param(**run_dict)

    if run.empty is False:
        run_path = run['run_path'].item()
#         run_metric = exp.read_metric(run_path, metric_name)
        ##################################
        # Fetch the data for each epoch
        data_list = []
        for epoch in range(1, num_epochs + 1):
            metric_name = f"epoch_{epoch}_cost_min"
            run_metric = exp.read_metric(run_path, metric_name)
            # print(run_metric.name)
            epoch_data = pd.DataFrame({
                'epoch': epoch,
                'step': np.arange(1, num_steps + 1),
                'model_path': run_metric.model_path,
                'value': run_metric.data  # assuming this is a list or array of length num_steps
            })
            data_list.append(epoch_data)

        # Concatenate all dataframes
        data = pd.concat(data_list)

        # calculate mean values for line plot
        min_values = data.groupby('epoch')['value'].min() # (self, data, model_path, problem, alpha=0.05, name=None):
        
        ##############
        run_metric_list.append(Metric(min_values, run_metric.model_path, problem , name=metric_name))
#         plot_dict[run_metric.model_path+ '-' +run_metric.name] = Metric(min_values.values, run_metric.model_path, problem , name=metric_name)
        plot_dict[run_metric.model_path+ '-' +run_metric.name] = min_values
    else:
        pdb.set_trace()
        print('Not found: ', run_dict)

data = pd.DataFrame(plot_dict)

# Create the boxplot using plotly.graph_objs
fig = make_subplots(rows=1, cols=1)

for idx, column in enumerate(data.columns):
    column_parts = column.split('-')
    column_name = column_parts[0]
    metric_name = column_parts[1]
    fig.add_trace(go.Box(y=data[column], name=name_dict[column_name], boxpoints='outliers', showlegend=True,
                         marker=dict(color=colors[idx], 
                                     line=dict(width=0.3),
#                                      outliercolor='rgba(219, 64, 82, 0.6)', # Optional: set color for outlier points
#                                      outliersize=6,
                                    )))

fig.update_layout(#title='Metric: '+ metric_name,
                  plot_bgcolor='rgb(255, 255, 255)',
                  width=800,
                  height=600,
                  xaxis=dict(
#                       title_text='Evaluation iterations',
                      gridcolor='rgb(200, 200, 200)',
                      zerolinecolor='rgb(200, 200, 200)'
                  ),
                    yaxis=dict(
                        title_text='Log10(Valor do Fitness)',
                        type="log",
                        gridcolor='rgb(200, 200, 200)',
                        zerolinecolor='rgb(200, 200, 200)',
#                         range=[log10(0.1), log10(400)]  # Adicione essa linha para focar em um subconjunto específico dos dados
                    ),
                  font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="black"
                  ),
                  showlegend=False,
                  legend=dict(
                      x=0,
                      y=1.005,
                      xanchor='left',
                      yanchor='bottom',
                      traceorder="normal",
                      font=dict(
                          family="sans-serif",
                          size=12,
                          color="black"
                      ),
                      bgcolor='rgba(229, 236, 246, 0)',
                      bordercolor="Black",
                      borderwidth=1
                  ))
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/square_cos_2d_boxplot.pdf', scale=10) #width=800, height=600, scale=2

In [ ]:
def pairwise_test(samples):
    n = len(samples)
    for i in range(n):
        for j in range(i+1, n):
            # print(f'Testing pair ({samples[i]}, {samples[j]})')
            test = StatTest('mannwhitneyu', samples[i].data, samples[j].data, name1 = samples[i].model_path, name2 = samples[j].model_path)
#             if test.result is True:
            print(samples[i])
            print(samples[j])
            print(test)
            print('\n')
            # replace this print statement with your actual test function
            # for example: test_result = test(samples[i], samples[j])
            # where "test" is your test function

# samples = ['A', 'B', 'C', 'D', 'E']
pairwise_test(run_metric_list)


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

problem = 'square_cos_2'

run_path_list = []
for (model_path, problem) in path_problem:
  run_dict = {'model_path': model_path,'problem': problem}
  # gambiarra pra plotar Adam
  if (run_dict['model_path']=='.\square_cos_2_Adam_exp1'): # and (run_dict['problem']=='square_cos_5'):
      run_dict['problem']='rastrigin'
  run = exp.get_run_by_param(**run_dict)
  if run.empty is False:
    run_path_list.append(run['run_path'].item())

# Assume you have the `exp` object available and the number of epochs
num_epochs = 100
num_steps = 250
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]


# colors = ['#1f77b4','#d62728','#ff7f0e', '#2ca02c','#ff7f0e','#9467bd','#8c564b','#e377c2', '#7f7f7f' ,'#bcbd22']

# Initialize the figure
fig = go.Figure()

# fetch and plot data for each optimizer
for opt_idx, run_path in enumerate(run_path_list):

    data_list = []

    # Fetch the data for each epoch
    for epoch in range(1, num_epochs + 1):
        metric_name = f"epoch_{epoch}_cost_min"
        run_metric = exp.read_metric(run_path, metric_name)
        # print(run_metric.name)
        epoch_data = pd.DataFrame({
            'epoch': epoch,
            'step': np.arange(1, num_steps + 1),
            'model_path': run_metric.model_path,
            'value': run_metric.data  # assuming this is a list or array of length num_steps
        })
        data_list.append(epoch_data)

    # Concatenate all dataframes
    data = pd.concat(data_list)
    # calculate mean values for line plot
#     mean_values = data.groupby('epoch')['value'].mean()
    # Agrupe por 'step' e calcule o mínimo para cada grupo
    mean_values = data.groupby('step')['value'].mean()
#     print(mean_values)

#     # create box plots for each 10th step
#     for step in range(1, num_steps+1):
#         if step % 5 == 0:  # only create boxplot for every 10th step
#             fig.add_trace(go.Box(
#                 y=data.loc[data['step'] == step, 'value'],
#                 x=[step] * len(data.loc[data['step'] == step, 'value']),  # specify x-position for each point in the box plot
#                 name=str(step),
#                 showlegend=False,
#                 boxpoints='outliers',#'outliers',
#                 width=0.3,  # Adjust this value to make the box narrower or wider
#                 marker=dict(
#                   size=0.5,
#                   color=colors[opt_idx][:3] + 'a'+ colors[opt_idx][3:-1]+',0.1)',  # use different color for each optimizer
#                   line=dict(width=0.3),  # change width of the box outline
#                 ),
#                 line=dict(width=0.3),  # change width of the whisker lines
#             ))

    # create line plot for the mean values
    fig.add_trace(go.Scatter(
        x=mean_values.index,  # numerical x-values
        y=mean_values.values,
        mode='lines',
        name=name_dict[run_metric.model_path],
        line=dict(
          color=colors[opt_idx],
          width=3,  # adjust this for line thickness
        ),
    ))


fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        type="log",
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=800,
    height=600,
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(255, 255, 255, 0.5)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=1
    )
)

fig.update_xaxes(
    title_text='Iterações',
    tickangle=0,
    tickvals=[i for i in range(0,261,50)],
    # range=[0, 150],
    )
fig.update_yaxes(
    title_text='log10(Valor do Fitness)',
#     range=[log10(0.05), log10(22)],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/square_cos_2_line_main.pdf', scale=10) #width=800, height=600, scale=2

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/square_cos_2_line_main_zoom1.pdf', scale=10) #width=800, height=600, scale=2



In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=350,
    height=500,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,1)],
    range=[246, 250],
    )
fig.update_yaxes(
    title_text='Loss',
    range=[0.125, 0.42],
    )
fig.show()

In [ ]:
# import plotly.io as pio

# pio.write_image(fig, '/content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23/quadratic_5_line_zoom2.pdf', scale=10) #width=800, height=600, scale=2

# from google.colab import files

# # Assuming you have a file named 'file_name.csv' in the current directory
# files.download('/content/drive/MyDrive/Projeto/Projeto2/Resultados/IECON23/quadratic_5_line_zoom2.pdf')

## Rosenbrock 5D

In [ ]:

path_problem = []
problem = 'rosenbrock_5'


# path_problem.append(('./rosenbrock_5_7p_only_im_finetuned_V2_exp1',problem))
# path_problem.append(('./weierstrass_5_7p_only_im_finetuned_V2_exp1_fixed',problem))

# path_problem.append(('./rosenbrock_5_7p_sumfx_im_V1_exp1',problem))
# path_problem.append(('./quadratic_5_7p_sumfx_im_V1_exp1',problem))


path_problem.append(('./quadratic_5_7p_baseline_exp3',problem))
path_problem.append(('./quadratic_5_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./square_cos_5_7p_baseline_exp2',problem))
path_problem.append(('./square_cos_5_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./rosenbrock_5_7p_baseline_exp1',problem))
path_problem.append(('./rosenbrock_5_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./weierstrass_5_7p_baseline_exp1',problem))
path_problem.append(('./weierstrass_5_7p_only_im_finetuned_V2_exp1_fixed',problem))

path_problem.append(('./ackley_5_7p_baseline_exp2',problem))
path_problem.append(('./ackley_5_7p_only_im_finetuned_V2_exp2',problem))

path_problem.append(('.\\rosenbrock_5_Adam_exp1',problem))
path_problem.append(('.\PSO',problem))

plot_dict = {}
metric_name = 'epoch_best_cost'
run_metric_list = []
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]


###########
num_epochs = 100
num_steps = 250
for idx, (model_path, problem) in enumerate(path_problem):
    run_dict = {'model_path':model_path,'problem':problem}
    if run_dict['model_path']=='.\\rosenbrock_5_Adam_exp1':
        run_dict['problem']='rosenbrock'
    run = exp.get_run_by_param(**run_dict)

    if run.empty is False:
        run_path = run['run_path'].item()
#         run_metric = exp.read_metric(run_path, metric_name)
        ##################################
        # Fetch the data for each epoch
        data_list = []
        for epoch in range(1, num_epochs + 1):
            metric_name = f"epoch_{epoch}_cost_min"
            run_metric = exp.read_metric(run_path, metric_name)
            # print(run_metric.name)
            epoch_data = pd.DataFrame({
                'epoch': epoch,
                'step': np.arange(1, num_steps + 1),
                'model_path': run_metric.model_path,
                'value': run_metric.data  # assuming this is a list or array of length num_steps
            })
            data_list.append(epoch_data)

        # Concatenate all dataframes
        data = pd.concat(data_list)

        # calculate mean values for line plot
        min_values = data.groupby('epoch')['value'].min() # (self, data, model_path, problem, alpha=0.05, name=None):
        
        ##############
        run_metric_list.append(Metric(min_values, run_metric.model_path, problem , name=metric_name))
#         plot_dict[run_metric.model_path+ '-' +run_metric.name] = Metric(min_values.values, run_metric.model_path, problem , name=metric_name)
        plot_dict[run_metric.model_path+ '-' +run_metric.name] = min_values
    else:
        pdb.set_trace()
        print('Not found: ', run_dict)

data = pd.DataFrame(plot_dict)

# Create the boxplot using plotly.graph_objs
fig = make_subplots(rows=1, cols=1)

for idx, column in enumerate(data.columns):
    column_parts = column.split('-')
    column_name = column_parts[0]
    metric_name = column_parts[1]
    fig.add_trace(go.Box(y=abs(data[column]), name=name_dict[column_name], boxpoints='outliers', showlegend=True,
                         marker=dict(color=colors[idx], 
                                     line=dict(width=0.3),
#                                      outliercolor='rgba(219, 64, 82, 0.6)', # Optional: set color for outlier points
#                                      outliersize=6,
                                    )))

fig.update_layout(#title='Metric: '+ metric_name,
                  plot_bgcolor='rgb(255, 255, 255)',
                  width=800,
                  height=600,
                  xaxis=dict(
#                       title_text='Evaluation iterations',
                      gridcolor='rgb(200, 200, 200)',
                      zerolinecolor='rgb(200, 200, 200)'
                  ),
                    yaxis=dict(
                        title_text='Log10(Valor do Fitness)',
                        type="log",
                        gridcolor='rgb(200, 200, 200)',
                        zerolinecolor='rgb(200, 200, 200)',
#                         range=[log10(0.1), log10(400)]  # Adicione essa linha para focar em um subconjunto específico dos dados
                    ),
                  font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="black"
                  ),
                  showlegend=False,
                  legend=dict(
                      x=0,
                      y=1.005,
                      xanchor='left',
                      yanchor='bottom',
                      traceorder="normal",
                      font=dict(
                          family="sans-serif",
                          size=12,
                          color="black"
                      ),
                      bgcolor='rgba(229, 236, 246, 0)',
                      bordercolor="Black",
                      borderwidth=1
                  ))
fig.show()

In [ ]:
pio.write_image(fig, './figures/rosenbrock_5d_boxplot.pdf', scale=10) #width=800, height=600, scale=2

In [ ]:
def pairwise_test(samples):
    n = len(samples)
    for i in range(n):
        for j in range(i+1, n):
            # print(f'Testing pair ({samples[i]}, {samples[j]})')
            test = StatTest('mannwhitneyu', samples[i].data, samples[j].data, name1 = samples[i].model_path, name2 = samples[j].model_path)
            if True: #test.result is False:
              print(samples[i])
              print(samples[j])
              print(test)
              print('\n')
            # replace this print statement with your actual test function
            # for example: test_result = test(samples[i], samples[j])
            # where "test" is your test function

# samples = ['A', 'B', 'C', 'D', 'E']
pairwise_test(run_metric_list)




In [ ]:
run_path_list = []
for (model_path, problem) in path_problem:
  run_dict = {'model_path': model_path,'problem': problem}
  # gambiarra pra plotar Adam
  if run_dict['model_path']=='.\\rosenbrock_5_Adam_exp1':
      run_dict['problem']='rosenbrock'
  run = exp.get_run_by_param(**run_dict)
  if run.empty is False:
    run_path_list.append(run['run_path'].item())

# Assume you have the `exp` object available and the number of epochs
num_epochs = 100
num_steps = 250
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]


# colors = ['#1f77b4','#d62728','#ff7f0e', '#2ca02c','#ff7f0e','#9467bd','#8c564b','#e377c2', '#7f7f7f' ,'#bcbd22']

# Initialize the figure
fig = go.Figure()

# fetch and plot data for each optimizer
for opt_idx, run_path in enumerate(run_path_list):

    data_list = []

    # Fetch the data for each epoch
    for epoch in range(1, num_epochs + 1):
        metric_name = f"epoch_{epoch}_cost_min"
        run_metric = exp.read_metric(run_path, metric_name)
        # print(run_metric.name)
        epoch_data = pd.DataFrame({
            'epoch': epoch,
            'step': np.arange(1, num_steps + 1),
            'model_path': run_metric.model_path,
            'value': run_metric.data  # assuming this is a list or array of length num_steps
        })
        data_list.append(epoch_data)

    # Concatenate all dataframes
    data = pd.concat(data_list)
    # calculate mean values for line plot
#     mean_values = data.groupby('epoch')['value'].mean()
    # Agrupe por 'step' e calcule o mínimo para cada grupo
    mean_values = data.groupby('step')['value'].mean()
#     print(mean_values)

#     # create box plots for each 10th step
#     for step in range(1, num_steps+1):
#         if step % 5 == 0:  # only create boxplot for every 10th step
#             fig.add_trace(go.Box(
#                 y=data.loc[data['step'] == step, 'value'],
#                 x=[step] * len(data.loc[data['step'] == step, 'value']),  # specify x-position for each point in the box plot
#                 name=str(step),
#                 showlegend=False,
#                 boxpoints='outliers',#'outliers',
#                 width=0.3,  # Adjust this value to make the box narrower or wider
#                 marker=dict(
#                   size=0.5,
#                   color=colors[opt_idx][:3] + 'a'+ colors[opt_idx][3:-1]+',0.1)',  # use different color for each optimizer
#                   line=dict(width=0.3),  # change width of the box outline
#                 ),
#                 line=dict(width=0.3),  # change width of the whisker lines
#             ))

    # create line plot for the mean values
    fig.add_trace(go.Scatter(
        x=mean_values.index,  # numerical x-values
        y=mean_values.values,
        mode='lines',
        name=name_dict[run_metric.model_path],
        line=dict(
          color=colors[opt_idx],
          width=3,  # adjust this for line thickness
        ),
    ))


fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        type="log",
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=800,
    height=600,
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(255, 255, 255, 0.5)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=1
    )
)

fig.update_xaxes(
    title_text='Iterações',
    tickangle=0,
    tickvals=[i for i in range(0,261,50)],
    # range=[0, 150],
    )
fig.update_yaxes(
    title_text='log10(Valor do Fitness)',
#     range=[log10(0.05), log10(22)],
    )
fig.show()



In [ ]:
pio.write_image(fig, './figures/rosenbrock_5_line_main.pdf', scale=10) #width=800, height=600, scale=2

## Rosenbrock 2d

In [ ]:
path_problem = []
problem = 'rosenbrock_2'



# path_problem.append(('./quadratic_2_7p_sumfx_im_V1_exp1',problem))

path_problem.append(('./quadratic_2_7p_baseline_exp3',problem))
path_problem.append(('./quadratic_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./square_cos_2_7p_baseline_exp1',problem))
path_problem.append(('./square_cos_2_7p_only_im_finetuned_V2_exp2',problem))

path_problem.append(('./rosenbrock_2_7p_baseline_exp2',problem))
path_problem.append(('./rosenbrock_2_7p_only_im_finetuned_V2_exp3',problem))

# path_problem.append(('./weierstrass_2_7p_baseline_exp1',problem))
path_problem.append(('./weierstrass_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./ackley_2_7p_baseline_exp1',problem))
path_problem.append(('./ackley_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('.\\rosenbrock_2_Adam_exp1',problem))
path_problem.append(('.\PSO',problem))



plot_dict = {}
metric_name = 'epoch_best_cost'
run_metric_list = []
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]


###########
num_epochs = 100
num_steps = 250
for idx, (model_path, problem) in enumerate(path_problem):
    run_dict = {'model_path':model_path,'problem':problem}
    if run_dict['model_path']=='.\\rosenbrock_2_Adam_exp1':
        run_dict['problem']='rosenbrock'
    run = exp.get_run_by_param(**run_dict)

    if run.empty is False:
        run_path = run['run_path'].item()
#         run_metric = exp.read_metric(run_path, metric_name)
        ##################################
        # Fetch the data for each epoch
        data_list = []
        for epoch in range(1, num_epochs + 1):
            metric_name = f"epoch_{epoch}_cost_min"
            run_metric = exp.read_metric(run_path, metric_name)
            # print(run_metric.name)
            epoch_data = pd.DataFrame({
                'epoch': epoch,
                'step': np.arange(1, num_steps + 1),
                'model_path': run_metric.model_path,
                'value': run_metric.data  # assuming this is a list or array of length num_steps
            })
            data_list.append(epoch_data)

        # Concatenate all dataframes
        data = pd.concat(data_list)

        # calculate mean values for line plot
        min_values = data.groupby('epoch')['value'].min() # (self, data, model_path, problem, alpha=0.05, name=None):
        
        ##############
        run_metric_list.append(Metric(min_values, run_metric.model_path, problem , name=metric_name))
#         plot_dict[run_metric.model_path+ '-' +run_metric.name] = Metric(min_values.values, run_metric.model_path, problem , name=metric_name)
        plot_dict[run_metric.model_path+ '-' +run_metric.name] = min_values
    else:
        pdb.set_trace()
        print('Not found: ', run_dict)

data = pd.DataFrame(plot_dict)

# Create the boxplot using plotly.graph_objs
fig = make_subplots(rows=1, cols=1)

for idx, column in enumerate(data.columns):
    column_parts = column.split('-')
    column_name = column_parts[0]
    metric_name = column_parts[1]
    fig.add_trace(go.Box(y=abs(data[column]), name=name_dict[column_name], boxpoints='outliers', showlegend=True,
                         marker=dict(color=colors[idx], 
                                     line=dict(width=0.3),
#                                      outliercolor='rgba(219, 64, 82, 0.6)', # Optional: set color for outlier points
#                                      outliersize=6,
                                    )))

fig.update_layout(#title='Metric: '+ metric_name,
                  plot_bgcolor='rgb(255, 255, 255)',
                  width=800,
                  height=600,
                  xaxis=dict(
#                       title_text='Evaluation iterations',
                      gridcolor='rgb(200, 200, 200)',
                      zerolinecolor='rgb(200, 200, 200)'
                  ),
                    yaxis=dict(
                        title_text='Log10(Valor do Fitness)',
                        type="log",
                        gridcolor='rgb(200, 200, 200)',
                        zerolinecolor='rgb(200, 200, 200)',
#                         range=[log10(0.1), log10(400)]  # Adicione essa linha para focar em um subconjunto específico dos dados
                    ),
                  font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="black"
                  ),
                  showlegend=False,
                  legend=dict(
                      x=0,
                      y=1.005,
                      xanchor='left',
                      yanchor='bottom',
                      traceorder="normal",
                      font=dict(
                          family="sans-serif",
                          size=12,
                          color="black"
                      ),
                      bgcolor='rgba(229, 236, 246, 0)',
                      bordercolor="Black",
                      borderwidth=1
                  ))
fig.show()



In [ ]:
pio.write_image(fig, './figures/rosenbrock_2d_boxplot.pdf', scale=10) #width=800, height=600, scale=2

In [ ]:
def pairwise_test(samples):
    n = len(samples)
    for i in range(n):
        for j in range(i+1, n):
            # print(f'Testing pair ({samples[i]}, {samples[j]})')
            test = StatTest('mannwhitneyu', samples[i].data, samples[j].data, name1 = samples[i].model_path, name2 = samples[j].model_path)
            if True: #test.result is False:
              print(samples[i])
              print(samples[j])
              print(test)
              print('\n')
            # replace this print statement with your actual test function
            # for example: test_result = test(samples[i], samples[j])
            # where "test" is your test function

# samples = ['A', 'B', 'C', 'D', 'E']
pairwise_test(run_metric_list)




In [ ]:
run_path_list = []
for (model_path, problem) in path_problem:
  run_dict = {'model_path': model_path,'problem': problem}
  # gambiarra pra plotar Adam
  if (run_dict['model_path']=='.\\rosenbrock_2_Adam_exp1'): # and (run_dict['problem']=='square_cos_5'):
      run_dict['problem']='rosenbrock'
  run = exp.get_run_by_param(**run_dict)
  if run.empty is False:
    run_path_list.append(run['run_path'].item())

# Assume you have the `exp` object available and the number of epochs
num_epochs = 100
num_steps = 250
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]


# colors = ['#1f77b4','#d62728','#ff7f0e', '#2ca02c','#ff7f0e','#9467bd','#8c564b','#e377c2', '#7f7f7f' ,'#bcbd22']

# Initialize the figure
fig = go.Figure()

# fetch and plot data for each optimizer
for opt_idx, run_path in enumerate(run_path_list):

    data_list = []

    # Fetch the data for each epoch
    for epoch in range(1, num_epochs + 1):
        metric_name = f"epoch_{epoch}_cost_min"
        run_metric = exp.read_metric(run_path, metric_name)
        # print(run_metric.name)
        epoch_data = pd.DataFrame({
            'epoch': epoch,
            'step': np.arange(1, num_steps + 1),
            'model_path': run_metric.model_path,
            'value': run_metric.data  # assuming this is a list or array of length num_steps
        })
        data_list.append(epoch_data)

    # Concatenate all dataframes
    data = pd.concat(data_list)
    # calculate mean values for line plot
#     mean_values = data.groupby('epoch')['value'].mean()
    # Agrupe por 'step' e calcule o mínimo para cada grupo
    mean_values = data.groupby('step')['value'].mean()
#     print(mean_values)

#     # create box plots for each 10th step
#     for step in range(1, num_steps+1):
#         if step % 5 == 0:  # only create boxplot for every 10th step
#             fig.add_trace(go.Box(
#                 y=data.loc[data['step'] == step, 'value'],
#                 x=[step] * len(data.loc[data['step'] == step, 'value']),  # specify x-position for each point in the box plot
#                 name=str(step),
#                 showlegend=False,
#                 boxpoints='outliers',#'outliers',
#                 width=0.3,  # Adjust this value to make the box narrower or wider
#                 marker=dict(
#                   size=0.5,
#                   color=colors[opt_idx][:3] + 'a'+ colors[opt_idx][3:-1]+',0.1)',  # use different color for each optimizer
#                   line=dict(width=0.3),  # change width of the box outline
#                 ),
#                 line=dict(width=0.3),  # change width of the whisker lines
#             ))

    # create line plot for the mean values
    fig.add_trace(go.Scatter(
        x=mean_values.index,  # numerical x-values
        y=mean_values.values,
        mode='lines',
        name=name_dict[run_metric.model_path],
        line=dict(
          color=colors[opt_idx],
          width=3,  # adjust this for line thickness
        ),
    ))


fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        type="log",
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=800,
    height=600,
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(255, 255, 255, 0.5)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=1
    )
)

fig.update_xaxes(
    title_text='Iterações',
    tickangle=0,
    tickvals=[i for i in range(0,261,50)],
    # range=[0, 150],
    )
fig.update_yaxes(
    title_text='log10(Valor do Fitness)',
#     range=[log10(0.05), log10(22)],
    )
fig.show()

In [ ]:
pio.write_image(fig, './figures/rosenbrock_2_line_main.pdf', scale=10) #width=800, height=600, scale=2

## Weierstrass 5d

In [ ]:
problem = 'weierstrass_5'

path_problem = []


# path_problem.append(('./quadratic_5_7p_sumfx_im_V1_exp1',problem))
path_problem.append(('./quadratic_5_7p_baseline_exp3',problem))
path_problem.append(('./quadratic_5_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./square_cos_5_7p_baseline_exp2',problem))
path_problem.append(('./square_cos_5_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./rosenbrock_5_7p_baseline_exp2',problem))
# path_problem.append(('./rosenbrock_5_7p_sumfx_im_V1_exp1',problem))
path_problem.append(('./rosenbrock_5_7p_only_im_finetuned_V2_exp2',problem))

path_problem.append(('./weierstrass_5_7p_baseline_exp1',problem))
path_problem.append(('./weierstrass_5_7p_only_im_finetuned_V2_exp1_fixed',problem))
# path_problem.append(('./rosenbrock_5_7p_baseline_exp3',problem))

path_problem.append(('./ackley_5_7p_baseline_exp2',problem))
path_problem.append(('./ackley_5_7p_only_im_finetuned_V2_exp1',problem))

# path_problem.append(('./square_cos_5_7p_baseline_exp3',problem))
path_problem.append(('.\weierstrass_5_Adam_exp1',problem))
path_problem.append(('.\PSO',problem))

plot_dict = {}
metric_name = 'epoch_best_cost'
run_metric_list = []
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]


###########
num_epochs = 100
num_steps = 250
for idx, (model_path, problem) in enumerate(path_problem):
    run_dict = {'model_path':model_path,'problem':problem}
    if run_dict['model_path']=='.\weierstrass_5_Adam_exp1':
        run_dict['problem']='weierstrass'
    run = exp.get_run_by_param(**run_dict)

    if run.empty is False:
        run_path = run['run_path'].item()
#         run_metric = exp.read_metric(run_path, metric_name)
        ##################################
        # Fetch the data for each epoch
        data_list = []
        for epoch in range(1, num_epochs + 1):
            metric_name = f"epoch_{epoch}_cost_min"
            run_metric = exp.read_metric(run_path, metric_name)
            # print(run_metric.name)
            epoch_data = pd.DataFrame({
                'epoch': epoch,
                'step': np.arange(1, num_steps + 1),
                'model_path': run_metric.model_path,
                'value': run_metric.data  # assuming this is a list or array of length num_steps
            })
            data_list.append(epoch_data)

        # Concatenate all dataframes
        data = pd.concat(data_list)

        # calculate mean values for line plot
        min_values = data.groupby('epoch')['value'].min() # (self, data, model_path, problem, alpha=0.05, name=None):
        
        ##############
        run_metric_list.append(Metric(min_values, run_metric.model_path, problem , name=metric_name))
#         plot_dict[run_metric.model_path+ '-' +run_metric.name] = Metric(min_values.values, run_metric.model_path, problem , name=metric_name)
        plot_dict[run_metric.model_path+ '-' +run_metric.name] = min_values
    else:
        pdb.set_trace()
        print('Not found: ', run_dict)

data = pd.DataFrame(plot_dict)

# Create the boxplot using plotly.graph_objs
fig = make_subplots(rows=1, cols=1)

for idx, column in enumerate(data.columns):
    column_parts = column.split('-')
    column_name = column_parts[0]
    metric_name = column_parts[1]
    fig.add_trace(go.Box(y=data[column], name=name_dict[column_name], boxpoints='outliers', showlegend=True,
                         marker=dict(color=colors[idx], 
                                     line=dict(width=0.3),
#                                      outliercolor='rgba(219, 64, 82, 0.6)', # Optional: set color for outlier points
#                                      outliersize=6,
                                    )))

fig.update_layout(#title='Metric: '+ metric_name,
                  plot_bgcolor='rgb(255, 255, 255)',
                  width=800,
                  height=600,
                  xaxis=dict(
#                       title_text='Evaluation iterations',
                      gridcolor='rgb(200, 200, 200)',
                      zerolinecolor='rgb(200, 200, 200)'
                  ),
                    yaxis=dict(
                        title_text='Log10(Valor do Fitness)',
                        type="log",
                        gridcolor='rgb(200, 200, 200)',
                        zerolinecolor='rgb(200, 200, 200)',
#                         range=[log10(0.1), log10(400)]  # Adicione essa linha para focar em um subconjunto específico dos dados
                    ),
                  font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="black"
                  ),
                  showlegend=False,
                  legend=dict(
                      x=0,
                      y=1.005,
                      xanchor='left',
                      yanchor='bottom',
                      traceorder="normal",
                      font=dict(
                          family="sans-serif",
                          size=12,
                          color="black"
                      ),
                      bgcolor='rgba(229, 236, 246, 0)',
                      bordercolor="Black",
                      borderwidth=1
                  ))
fig.show()


In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/weierstrass_5d_boxplot.pdf', scale=10) #width=800, height=600, scale=2

### Weierstrass 5d - Statistic Tests 

In [ ]:
def pairwise_test(samples):
    n = len(samples)
    for i in range(n):
        for j in range(i+1, n):
            # print(f'Testing pair ({samples[i]}, {samples[j]})')
            test = StatTest('mannwhitneyu', samples[i].data, samples[j].data, name1 = samples[i].model_path, name2 = samples[j].model_path)
            if True: #test.result is False:
              print(samples[i])
              print(samples[j])
              print(test)
              print('\n')
            # replace this print statement with your actual test function
            # for example: test_result = test(samples[i], samples[j])
            # where "test" is your test function

# samples = ['A', 'B', 'C', 'D', 'E']
pairwise_test(run_metric_list)




In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

problem = 'weierstrass_5'

run_path_list = []
for (model_path, problem) in path_problem:
  run_dict = {'model_path': model_path,'problem': problem}
  # gambiarra pra plotar Adam
  if run_dict['model_path']=='.\weierstrass_5_Adam_exp1':
      run_dict['problem']='weierstrass'
  run = exp.get_run_by_param(**run_dict)
  if run.empty is False:
    run_path_list.append(run['run_path'].item())

# Assume you have the `exp` object available and the number of epochs
num_epochs = 100
num_steps = 250
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]

# colors = ['#1f77b4','#d62728','#ff7f0e', '#2ca02c','#ff7f0e','#9467bd','#8c564b','#e377c2', '#7f7f7f' ,'#bcbd22']

###########
# Initialize the figure
fig = go.Figure()

# fetch and plot data for each optimizer
for opt_idx, run_path in enumerate(run_path_list):

    data_list = []

    # Fetch the data for each epoch
    for epoch in range(1, num_epochs + 1):
        metric_name = f"epoch_{epoch}_cost_min"
        run_metric = exp.read_metric(run_path, metric_name)
        # print(run_metric.name)
        epoch_data = pd.DataFrame({
            'epoch': epoch,
            'step': np.arange(1, num_steps + 1),
            'model_path': run_metric.model_path,
            'value': run_metric.data  # assuming this is a list or array of length num_steps
        })
        data_list.append(epoch_data)

    # Concatenate all dataframes
    data = pd.concat(data_list)
    # calculate mean values for line plot
#     mean_values = data.groupby('epoch')['value'].mean()
    # Agrupe por 'step' e calcule o mínimo para cada grupo
    mean_values = data.groupby('step')['value'].mean()
#     print(mean_values)

#     # create box plots for each 10th step
#     for step in range(1, num_steps+1):
#         if step % 5 == 0:  # only create boxplot for every 10th step
#             fig.add_trace(go.Box(
#                 y=data.loc[data['step'] == step, 'value'],
#                 x=[step] * len(data.loc[data['step'] == step, 'value']),  # specify x-position for each point in the box plot
#                 name=str(step),
#                 showlegend=False,
#                 boxpoints='outliers',#'outliers',
#                 width=0.3,  # Adjust this value to make the box narrower or wider
#                 marker=dict(
#                   size=0.5,
#                   color=colors[opt_idx][:3] + 'a'+ colors[opt_idx][3:-1]+',0.1)',  # use different color for each optimizer
#                   line=dict(width=0.3),  # change width of the box outline
#                 ),
#                 line=dict(width=0.3),  # change width of the whisker lines
#             ))

    # create line plot for the mean values
    fig.add_trace(go.Scatter(
        x=mean_values.index,  # numerical x-values
        y=mean_values.values,
        mode='lines',
        name=name_dict[run_metric.model_path],
        line=dict(
          color=colors[opt_idx],
          width=3,  # adjust this for line thickness
        ),
    ))


fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        type="log",
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=800,
    height=600,
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(255, 255, 255, 0.5)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=1
    )
)

fig.update_xaxes(
    title_text='Iterações',
    tickangle=0,
    tickvals=[i for i in range(0,261,50)],
    # range=[0, 150],
    )
fig.update_yaxes(
    title_text='log10(Valor do Fitness)',
#     range=[log10(0.05), log10(22)],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/weierstrass_5_line_main.pdf', scale=10) #width=800, height=600, scale=2


In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=400,
    height=500,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,10)],
    range=[0, 50],
    )
fig.update_yaxes(
    title_text='Loss',
    range=[0.2, 1.6],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/weierstrass_5_line_zoom1.pdf', scale=10) #width=800, height=600, scale=2


In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=350,
    height=500,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,1)],
    range=[246, 250],
    )
fig.update_yaxes(
    title_text='Loss',
    range=[0.125, 0.42],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/weierstrass_5_line_zoom2.pdf', scale=10) #width=800, height=600, scale=2

## Weierstrass 2d

In [ ]:
problem = 'weierstrass_2'

path_problem = []


# path_problem.append(('./quadratic_2_7p_sumfx_im_V1_exp1',problem))

path_problem.append(('./quadratic_2_7p_baseline_exp3',problem))
path_problem.append(('./quadratic_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./square_cos_2_7p_baseline_exp1',problem))
path_problem.append(('./square_cos_2_7p_only_im_finetuned_V2_exp2',problem))

path_problem.append(('./rosenbrock_2_7p_baseline_exp2',problem))
path_problem.append(('./rosenbrock_2_7p_only_im_finetuned_V2_exp3',problem))

path_problem.append(('./weierstrass_2_7p_baseline_exp1',problem))
path_problem.append(('./weierstrass_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./ackley_2_7p_baseline_exp1',problem))
path_problem.append(('./ackley_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('.\weierstrass_2_Adam_exp1',problem))
path_problem.append(('.\PSO',problem))


plot_dict = {}
metric_name = 'epoch_best_cost'
run_metric_list = []
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]


###########
num_epochs = 100
num_steps = 250
for idx, (model_path, problem) in enumerate(path_problem):
    run_dict = {'model_path':model_path,'problem':problem}
    if run_dict['model_path']=='.\weierstrass_2_Adam_exp1':
        run_dict['problem']='weierstrass'
    run = exp.get_run_by_param(**run_dict)

    if run.empty is False:
        run_path = run['run_path'].item()
#         run_metric = exp.read_metric(run_path, metric_name)
        ##################################
        # Fetch the data for each epoch
        data_list = []
        for epoch in range(1, num_epochs + 1):
            metric_name = f"epoch_{epoch}_cost_min"
            run_metric = exp.read_metric(run_path, metric_name)
            # print(run_metric.name)
            epoch_data = pd.DataFrame({
                'epoch': epoch,
                'step': np.arange(1, num_steps + 1),
                'model_path': run_metric.model_path,
                'value': run_metric.data  # assuming this is a list or array of length num_steps
            })
            data_list.append(epoch_data)

        # Concatenate all dataframes
        data = pd.concat(data_list)

        # calculate mean values for line plot
        min_values = data.groupby('epoch')['value'].min() # (self, data, model_path, problem, alpha=0.05, name=None):
        
        ##############
        run_metric_list.append(Metric(min_values, run_metric.model_path, problem , name=metric_name))
#         plot_dict[run_metric.model_path+ '-' +run_metric.name] = Metric(min_values.values, run_metric.model_path, problem , name=metric_name)
        plot_dict[run_metric.model_path+ '-' +run_metric.name] = min_values
    else:
        pdb.set_trace()
        print('Not found: ', run_dict)

data = pd.DataFrame(plot_dict)

# Create the boxplot using plotly.graph_objs
fig = make_subplots(rows=1, cols=1)

for idx, column in enumerate(data.columns):
    column_parts = column.split('-')
    column_name = column_parts[0]
    metric_name = column_parts[1]
    fig.add_trace(go.Box(y=abs(data[column]), name=name_dict[column_name], boxpoints='outliers', showlegend=True,
                         marker=dict(color=colors[idx], 
                                     line=dict(width=0.3),
#                                      outliercolor='rgba(219, 64, 82, 0.6)', # Optional: set color for outlier points
#                                      outliersize=6,
                                    )))

fig.update_layout(#title='Metric: '+ metric_name,
                  plot_bgcolor='rgb(255, 255, 255)',
                  width=800,
                  height=600,
                  xaxis=dict(
#                       title_text='Evaluation iterations',
                      gridcolor='rgb(200, 200, 200)',
                      zerolinecolor='rgb(200, 200, 200)'
                  ),
                    yaxis=dict(
                        title_text='Log10(Valor do Fitness)',
                        type="log",
                        gridcolor='rgb(200, 200, 200)',
                        zerolinecolor='rgb(200, 200, 200)',
#                         range=[log10(0.000000001), log10(12)]  # Adicione essa linha para focar em um subconjunto específico dos dados
                    ),
                  font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="black"
                  ),
                  showlegend=False,
                  legend=dict(
                      x=0,
                      y=1.005,
                      xanchor='left',
                      yanchor='bottom',
                      traceorder="normal",
                      font=dict(
                          family="sans-serif",
                          size=12,
                          color="black"
                      ),
                      bgcolor='rgba(229, 236, 246, 0)',
                      bordercolor="Black",
                      borderwidth=1
                  ))
fig.show()


In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/weierstrass_2d_boxplot.pdf', scale=10) #width=800, height=600, scale=2

### Statistic Tests 

In [ ]:
def pairwise_test(samples):
    n = len(samples)
    for i in range(n):
        for j in range(i+1, n):
            # print(f'Testing pair ({samples[i]}, {samples[j]})')
            test = StatTest('mannwhitneyu', samples[i].data, samples[j].data, name1 = samples[i].model_path, name2 = samples[j].model_path)
            if True: #test.result is False:
              print(samples[i])
              print(samples[j])
              print(test)
              print('\n')
            # replace this print statement with your actual test function
            # for example: test_result = test(samples[i], samples[j])
            # where "test" is your test function

# samples = ['A', 'B', 'C', 'D', 'E']
pairwise_test(run_metric_list)




In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

problem = 'weierstrass_2'

run_path_list = []
for (model_path, problem) in path_problem:
  run_dict = {'model_path': model_path,'problem': problem}
  if run_dict['model_path']=='.\weierstrass_2_Adam_exp1':
      run_dict['problem']='weierstrass'
  run = exp.get_run_by_param(**run_dict)
  if run.empty is False:
    run_path_list.append(run['run_path'].item())

# Assume you have the `exp` object available and the number of epochs
num_epochs = 100
num_steps = 250
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]

# colors = ['#1f77b4','#d62728','#ff7f0e', '#2ca02c','#ff7f0e','#9467bd','#8c564b','#e377c2', '#7f7f7f' ,'#bcbd22']

# Initialize the figure
fig = go.Figure()

# fetch and plot data for each optimizer
for opt_idx, run_path in enumerate(run_path_list):

    data_list = []

    # Fetch the data for each epoch
    for epoch in range(1, num_epochs + 1):
        metric_name = f"epoch_{epoch}_cost_min"
        run_metric = exp.read_metric(run_path, metric_name)
        # print(run_metric.name)
        epoch_data = pd.DataFrame({
            'epoch': epoch,
            'step': np.arange(1, num_steps + 1),
            'model_path': run_metric.model_path,
            'value': run_metric.data  # assuming this is a list or array of length num_steps
        })
        data_list.append(epoch_data)

    # Concatenate all dataframes
    data = pd.concat(data_list)
    # calculate mean values for line plot
#     mean_values = data.groupby('epoch')['value'].mean()
    # Agrupe por 'step' e calcule o mínimo para cada grupo
    mean_values = data.groupby('step')['value'].mean()
#     print(mean_values)

#     # create box plots for each 10th step
#     for step in range(1, num_steps+1):
#         if step % 5 == 0:  # only create boxplot for every 10th step
#             fig.add_trace(go.Box(
#                 y=data.loc[data['step'] == step, 'value'],
#                 x=[step] * len(data.loc[data['step'] == step, 'value']),  # specify x-position for each point in the box plot
#                 name=str(step),
#                 showlegend=False,
#                 boxpoints='outliers',#'outliers',
#                 width=0.3,  # Adjust this value to make the box narrower or wider
#                 marker=dict(
#                   size=0.5,
#                   color=colors[opt_idx][:3] + 'a'+ colors[opt_idx][3:-1]+',0.1)',  # use different color for each optimizer
#                   line=dict(width=0.3),  # change width of the box outline
#                 ),
#                 line=dict(width=0.3),  # change width of the whisker lines
#             ))

    # create line plot for the mean values
    fig.add_trace(go.Scatter(
        x=mean_values.index,  # numerical x-values
        y=mean_values.values,
        mode='lines',
        name=name_dict[run_metric.model_path],
        line=dict(
          color=colors[opt_idx],
          width=3,  # adjust this for line thickness
        ),
    ))


fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        type="log",
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=800,
    height=600,
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(255, 255, 255, 0.5)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=1
    )
)

fig.update_xaxes(
    title_text='Iterações',
    tickangle=0,
    tickvals=[i for i in range(0,261,50)],
    # range=[0, 150],
    )
fig.update_yaxes(
    title_text='log10(Valor do Fitness)',
#     range=[log10(0.05), log10(22)],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/weierstrass_2_line_main.pdf', scale=10) #width=800, height=600, scale=2


In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=400,
    height=500,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,10)],
    range=[0, 50],
    )
fig.update_yaxes(
    title_text='Loss',
    range=[0.2, 1.6],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/weierstrass_2_line_zoom1.pdf', scale=10) #width=800, height=600, scale=2


In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=350,
    height=500,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,1)],
    range=[246, 250],
    )
fig.update_yaxes(
    title_text='Loss',
    range=[0.125, 0.42],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/weierstrass_5_line_zoom2.pdf', scale=10) #width=800, height=600, scale=2

## Ackley 5d

In [ ]:
problem = 'ackley_5'

path_problem = []

# path_problem.append(('./ackley_5_7p_only_im_finetuned_V2_exp1',problem))


# path_problem.append(('./rosenbrock_5_7p_sumfx_im_V1_exp1',problem))
# path_problem.append(('./quadratic_5_7p_sumfx_im_V1_exp1',problem))

path_problem.append(('./quadratic_5_7p_baseline_exp3',problem))
path_problem.append(('./quadratic_5_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./square_cos_5_7p_baseline_exp2',problem))
path_problem.append(('./square_cos_5_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./rosenbrock_5_7p_baseline_exp1',problem))
path_problem.append(('./rosenbrock_5_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./weierstrass_5_7p_baseline_exp1',problem))
path_problem.append(('./weierstrass_5_7p_only_im_finetuned_V2_exp1_fixed',problem))

path_problem.append(('./ackley_5_7p_baseline_exp1',problem))
path_problem.append(('./ackley_5_7p_only_im_finetuned_V2_exp1',problem))
# path_problem.append(('./rosenbrock_5_7p_baseline_exp3',problem))

# path_problem.append(('./square_cos_5_7p_baseline_exp3',problem))
path_problem.append(('.\ackley_5_Adam_exp1',problem))
path_problem.append(('.\PSO',problem))

plot_dict = {}
metric_name = 'epoch_best_cost'
run_metric_list = []
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]

num_epochs = 100
num_steps = 250
for idx, (model_path, problem) in enumerate(path_problem):
    run_dict = {'model_path':model_path,'problem':problem}
    #   if run_dict['model_path']=='.\square_cos_5_Adam_exp1':
    #       run_dict['problem']='rastrigin'
    #   if run_dict['model_path']=='.\square_cos_5_PSO':
    #       run_dict['problem']='quadratic'
    run = exp.get_run_by_param(**run_dict)

    if run.empty is False:
        run_path = run['run_path'].item()
#         run_metric = exp.read_metric(run_path, metric_name)
        ##################################
        # Fetch the data for each epoch
        data_list = []
        for epoch in range(1, num_epochs + 1):
            metric_name = f"epoch_{epoch}_cost_min"
            run_metric = exp.read_metric(run_path, metric_name)
            # print(run_metric.name)
            epoch_data = pd.DataFrame({
                'epoch': epoch,
                'step': np.arange(1, num_steps + 1),
                'model_path': run_metric.model_path,
                'value': run_metric.data  # assuming this is a list or array of length num_steps
            })
            data_list.append(epoch_data)

        # Concatenate all dataframes
        data = pd.concat(data_list)

        # calculate mean values for line plot
        min_values = data.groupby('epoch')['value'].min() # (self, data, model_path, problem, alpha=0.05, name=None):
        
        ##############
        run_metric_list.append(Metric(min_values, run_metric.model_path, problem , name=metric_name))
#         plot_dict[run_metric.model_path+ '-' +run_metric.name] = Metric(min_values.values, run_metric.model_path, problem , name=metric_name)
        plot_dict[run_metric.model_path+ '-' +run_metric.name] = min_values
    else:
#         pdb.set_trace()
        print('Not found: ', run_dict)

data = pd.DataFrame(plot_dict)

# Create the boxplot using plotly.graph_objs
fig = make_subplots(rows=1, cols=1)

for idx, column in enumerate(data.columns):
    column_parts = column.split('-')
    column_name = column_parts[0]
    metric_name = column_parts[1]
    fig.add_trace(go.Box(y=data[column], name=name_dict[column_name], boxpoints='outliers', showlegend=True,
                         marker=dict(color=colors[idx], 
                                     line=dict(width=0.3),
#                                      outliercolor='rgba(219, 64, 82, 0.6)', # Optional: set color for outlier points
#                                      outliersize=6,
                                    )))

fig.update_layout(#title='Metric: '+ metric_name,
                  plot_bgcolor='rgb(255, 255, 255)',
                  width=800,
                  height=600,
                  xaxis=dict(
#                       title_text='Evaluation iterations',
                      gridcolor='rgb(200, 200, 200)',
                      zerolinecolor='rgb(200, 200, 200)'
                  ),
                    yaxis=dict(
                        title_text='Log10(Valor do Fitness)',
                        type="log",
                        gridcolor='rgb(200, 200, 200)',
                        zerolinecolor='rgb(200, 200, 200)',
#                         range=[log10(0.1), log10(400)]  # Adicione essa linha para focar em um subconjunto específico dos dados
                    ),
                  font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="black"
                  ),
                  showlegend=False,
                  legend=dict(
                      x=0,
                      y=1.005,
                      xanchor='left',
                      yanchor='bottom',
                      traceorder="normal",
                      font=dict(
                          family="sans-serif",
                          size=12,
                          color="black"
                      ),
                      bgcolor='rgba(229, 236, 246, 0)',
                      bordercolor="Black",
                      borderwidth=1
                  ))
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/ackley_5d_boxplot.pdf', scale=10) #width=800, height=600, scale=2

### Ackley 5d - Statistic Tests 

In [ ]:
def pairwise_test(samples):
    n = len(samples)
    for i in range(n):
        for j in range(i+1, n):
            # print(f'Testing pair ({samples[i]}, {samples[j]})')
            test = StatTest('mannwhitneyu', samples[i].data, samples[j].data, name1 = samples[i].model_path, name2 = samples[j].model_path)
            if test.result is False:
              print(samples[i])
              print(samples[j])
              print(test)
              print('\n')
            # replace this print statement with your actual test function
            # for example: test_result = test(samples[i], samples[j])
            # where "test" is your test function

# samples = ['A', 'B', 'C', 'D', 'E']
pairwise_test(run_metric_list)




In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

problem = 'ackley_5'

run_path_list = []
for (model_path, problem) in path_problem:
  run_dict = {'model_path': model_path,'problem': problem}
  # gambiarra pra plotar Adam
  # if (run_dict['model_path']=='.\square_cos_5_Adam_exp1'): # and (run_dict['problem']=='square_cos_5'):
  #     run_dict['problem']='rastrigin'
  run = exp.get_run_by_param(**run_dict)
  if run.empty is False:
    run_path_list.append(run['run_path'].item())

# Assume you have the `exp` object available and the number of epochs
num_epochs = 100
num_steps = 250
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]

# colors = ['#1f77b4','#d62728','#ff7f0e', '#2ca02c','#ff7f0e','#9467bd','#8c564b','#e377c2', '#7f7f7f' ,'#bcbd22']

# Initialize the figure
fig = go.Figure()

# fetch and plot data for each optimizer
for opt_idx, run_path in enumerate(run_path_list):

    data_list = []

    # Fetch the data for each epoch
    for epoch in range(1, num_epochs + 1):
        metric_name = f"epoch_{epoch}_cost_min"
        run_metric = exp.read_metric(run_path, metric_name)
        # print(run_metric.name)
        epoch_data = pd.DataFrame({
            'epoch': epoch,
            'step': np.arange(1, num_steps + 1),
            'model_path': run_metric.model_path,
            'value': run_metric.data  # assuming this is a list or array of length num_steps
        })
        data_list.append(epoch_data)

    # Concatenate all dataframes
    data = pd.concat(data_list)
    # calculate mean values for line plot
#     mean_values = data.groupby('epoch')['value'].mean()
    # Agrupe por 'step' e calcule o mínimo para cada grupo
    mean_values = data.groupby('step')['value'].mean()
#     print(mean_values)

#     # create box plots for each 10th step
#     for step in range(1, num_steps+1):
#         if step % 5 == 0:  # only create boxplot for every 10th step
#             fig.add_trace(go.Box(
#                 y=data.loc[data['step'] == step, 'value'],
#                 x=[step] * len(data.loc[data['step'] == step, 'value']),  # specify x-position for each point in the box plot
#                 name=str(step),
#                 showlegend=False,
#                 boxpoints='outliers',#'outliers',
#                 width=0.3,  # Adjust this value to make the box narrower or wider
#                 marker=dict(
#                   size=0.5,
#                   color=colors[opt_idx][:3] + 'a'+ colors[opt_idx][3:-1]+',0.1)',  # use different color for each optimizer
#                   line=dict(width=0.3),  # change width of the box outline
#                 ),
#                 line=dict(width=0.3),  # change width of the whisker lines
#             ))

    # create line plot for the mean values
    fig.add_trace(go.Scatter(
        x=mean_values.index,  # numerical x-values
        y=mean_values.values,
        mode='lines',
        name=name_dict[run_metric.model_path],
        line=dict(
          color=colors[opt_idx],
          width=3,  # adjust this for line thickness
        ),
    ))


fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        type="log",
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=800,
    height=600,
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(255, 255, 255, 0.5)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=1
    )
)

fig.update_xaxes(
    title_text='Iterações',
    tickangle=0,
    tickvals=[i for i in range(0,261,50)],
    # range=[0, 150],
    )
fig.update_yaxes(
    title_text='Valor do Fitness',
#     range=[log10(0.05), log10(22)],
#     range=[0, 1],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/ackley_5_line_main.pdf', scale=10) #width=800, height=600, scale=2


In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=400,
    height=500,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,10)],
    range=[0, 50],
    )
fig.update_yaxes(
    title_text='Loss',
    range=[0.2, 1.6],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/ackley_5_line_zoom1.pdf', scale=10) #width=800, height=600, scale=2


In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=350,
    height=500,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,1)],
    range=[246, 250],
    )
fig.update_yaxes(
    title_text='Loss',
    range=[0.125, 0.42],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/ackley_5_line_zoom2.pdf', scale=10) #width=800, height=600, scale=2

## Ackley 2d

In [ ]:
problem = 'ackley_2'

path_problem = []

path_problem.append(('./quadratic_2_7p_baseline_exp3',problem))
path_problem.append(('./quadratic_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./square_cos_2_7p_baseline_exp1',problem))
path_problem.append(('./square_cos_2_7p_only_im_finetuned_V2_exp2',problem))

path_problem.append(('./rosenbrock_2_7p_baseline_exp1',problem))
path_problem.append(('./rosenbrock_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('./weierstrass_2_7p_baseline_exp1',problem))
path_problem.append(('./weierstrass_2_7p_only_im_finetuned_V2_exp4',problem))

path_problem.append(('./ackley_2_7p_baseline_exp2',problem))
path_problem.append(('./ackley_2_7p_only_im_finetuned_V2_exp1',problem))

path_problem.append(('.\\ackley_2_Adam_exp1',problem))
path_problem.append(('.\PSO',problem))


plot_dict = {}
metric_name = 'epoch_best_cost'
run_metric_list = []
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]



######
num_epochs = 100
num_steps = 250
for idx, (model_path, problem) in enumerate(path_problem):
    run_dict = {'model_path':model_path,'problem':problem}
    #   if run_dict['model_path']=='.\square_cos_5_Adam_exp1':
    #       run_dict['problem']='rastrigin'
    #   if run_dict['model_path']=='.\square_cos_5_PSO':
    #       run_dict['problem']='quadratic'
    run = exp.get_run_by_param(**run_dict)

    if run.empty is False:
        run_path = run['run_path'].item()
#         run_metric = exp.read_metric(run_path, metric_name)
        ##################################
        # Fetch the data for each epoch
        data_list = []
        for epoch in range(1, num_epochs + 1):
            metric_name = f"epoch_{epoch}_cost_min"
            run_metric = exp.read_metric(run_path, metric_name)
            # print(run_metric.name)
            epoch_data = pd.DataFrame({
                'epoch': epoch,
                'step': np.arange(1, num_steps + 1),
                'model_path': run_metric.model_path,
                'value': run_metric.data  # assuming this is a list or array of length num_steps
            })
            data_list.append(epoch_data)

        # Concatenate all dataframes
        data = pd.concat(data_list)

        # calculate mean values for line plot
        min_values = data.groupby('epoch')['value'].min() # (self, data, model_path, problem, alpha=0.05, name=None):
        
        ##############
        run_metric_list.append(Metric(min_values, run_metric.model_path, problem , name=metric_name))
#         plot_dict[run_metric.model_path+ '-' +run_metric.name] = Metric(min_values.values, run_metric.model_path, problem , name=metric_name)
        plot_dict[run_metric.model_path+ '-' +run_metric.name] = min_values
    else:
#         pdb.set_trace()
        print('Not found: ', run_dict)

data = pd.DataFrame(plot_dict)

# Create the boxplot using plotly.graph_objs
fig = make_subplots(rows=1, cols=1)

for idx, column in enumerate(data.columns):
    column_parts = column.split('-')
    column_name = column_parts[0]
    metric_name = column_parts[1]
    fig.add_trace(go.Box(y=data[column], name=name_dict[column_name], boxpoints='outliers', showlegend=True,
                         marker=dict(color=colors[idx], 
                                     line=dict(width=0.3),
#                                      outliercolor='rgba(219, 64, 82, 0.6)', # Optional: set color for outlier points
#                                      outliersize=6,
                                    )))

fig.update_layout(#title='Metric: '+ metric_name,
                  plot_bgcolor='rgb(255, 255, 255)',
                  width=800,
                  height=600,
                  xaxis=dict(
#                       title_text='Evaluation iterations',
                      gridcolor='rgb(200, 200, 200)',
                      zerolinecolor='rgb(200, 200, 200)'
                  ),
                    yaxis=dict(
                        title_text='Log10(Valor do Fitness)',
                        type="log",
                        gridcolor='rgb(200, 200, 200)',
                        zerolinecolor='rgb(200, 200, 200)',
#                         range=[log10(0.1), log10(400)]  # Adicione essa linha para focar em um subconjunto específico dos dados
                    ),
                  font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="black"
                  ),
                  showlegend=False,
                  legend=dict(
                      x=0,
                      y=1.005,
                      xanchor='left',
                      yanchor='bottom',
                      traceorder="normal",
                      font=dict(
                          family="sans-serif",
                          size=12,
                          color="black"
                      ),
                      bgcolor='rgba(229, 236, 246, 0)',
                      bordercolor="Black",
                      borderwidth=1
                  ))
fig.show()


In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/ackley_2d_boxplot.pdf', scale=10) #width=800, height=600, scale=2

### Weierstrass 2d - Statistic Tests 

In [ ]:
def pairwise_test(samples):
    n = len(samples)
    for i in range(n):
        for j in range(i+1, n):
            # print(f'Testing pair ({samples[i]}, {samples[j]})')
            test = StatTest('mannwhitneyu', samples[i].data, samples[j].data, name1 = samples[i].model_path, name2 = samples[j].model_path)
            if True: #test.result is False:
              print(samples[i])
              print(samples[j])
              print(test)
              print('\n')
            # replace this print statement with your actual test function
            # for example: test_result = test(samples[i], samples[j])
            # where "test" is your test function

# samples = ['A', 'B', 'C', 'D', 'E']
pairwise_test(run_metric_list)




In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

problem = 'ackley_2'

run_path_list = []
for (model_path, problem) in path_problem:
  run_dict = {'model_path': model_path,'problem': problem}
  # gambiarra pra plotar Adam
  # if (run_dict['model_path']=='.\square_cos_5_Adam_exp1'): # and (run_dict['problem']=='square_cos_5'):
  #     run_dict['problem']='rastrigin'
  run = exp.get_run_by_param(**run_dict)
  if run.empty is False:
    run_path_list.append(run['run_path'].item())

# Assume you have the `exp` object available and the number of epochs
num_epochs = 100
num_steps = 250
# colors = [
#     'rgb(0, 114, 178)',
#     'rgb(230, 159, 0)',
#     'rgb(0, 158, 115)',
#     'rgb(86, 180, 233)',
#     'rgb(213, 94, 0)',
#     'rgb(204, 121, 167)',
#     'rgb(214, 39, 40)',
#     'rgb(148, 103, 189)',
#     'rgb(128, 128, 128)',
#     'rgb(240, 228, 66)',
# ]

# colors = ['#1f77b4','#d62728','#ff7f0e', '#2ca02c','#ff7f0e','#9467bd','#8c564b','#e377c2', '#7f7f7f' ,'#bcbd22']

# Initialize the figure
fig = go.Figure()

# fetch and plot data for each optimizer
for opt_idx, run_path in enumerate(run_path_list):

    data_list = []

    # Fetch the data for each epoch
    for epoch in range(1, num_epochs + 1):
        metric_name = f"epoch_{epoch}_cost_min"
        run_metric = exp.read_metric(run_path, metric_name)
        # print(run_metric.name)
        epoch_data = pd.DataFrame({
            'epoch': epoch,
            'step': np.arange(1, num_steps + 1),
            'model_path': run_metric.model_path,
            'value': run_metric.data  # assuming this is a list or array of length num_steps
        })
        data_list.append(epoch_data)

    # Concatenate all dataframes
    data = pd.concat(data_list)
    # calculate mean values for line plot
#     mean_values = data.groupby('epoch')['value'].mean()
    # Agrupe por 'step' e calcule o mínimo para cada grupo
    mean_values = data.groupby('step')['value'].mean()
#     print(mean_values)

#     # create box plots for each 10th step
#     for step in range(1, num_steps+1):
#         if step % 5 == 0:  # only create boxplot for every 10th step
#             fig.add_trace(go.Box(
#                 y=data.loc[data['step'] == step, 'value'],
#                 x=[step] * len(data.loc[data['step'] == step, 'value']),  # specify x-position for each point in the box plot
#                 name=str(step),
#                 showlegend=False,
#                 boxpoints='outliers',#'outliers',
#                 width=0.3,  # Adjust this value to make the box narrower or wider
#                 marker=dict(
#                   size=0.5,
#                   color=colors[opt_idx][:3] + 'a'+ colors[opt_idx][3:-1]+',0.1)',  # use different color for each optimizer
#                   line=dict(width=0.3),  # change width of the box outline
#                 ),
#                 line=dict(width=0.3),  # change width of the whisker lines
#             ))

    # create line plot for the mean values
    fig.add_trace(go.Scatter(
        x=mean_values.index,  # numerical x-values
        y=mean_values.values,
        mode='lines',
        name=name_dict[run_metric.model_path],
        line=dict(
          color=colors[opt_idx],
          width=3,  # adjust this for line thickness
        ),
    ))


fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        type="log",
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=800,
    height=600,
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(255, 255, 255, 0.5)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=1
    )
)

fig.update_xaxes(
    title_text='Iterações',
    tickangle=0,
    tickvals=[i for i in range(0,261,50)],
    # range=[0, 150],
    )
fig.update_yaxes(
    title_text='Valor do Fitness',
#     range=[log10(0.05), log10(22)],
#     range=[0, 1],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/ackley_2_line_main.pdf', scale=10) #width=800, height=600, scale=2


In [ ]:
# Initialize the figure
fig = go.Figure()

# fetch and plot data for each optimizer
for opt_idx, run_path in enumerate(run_path_list):

    data_list = []

    # Fetch the data for each epoch
    for epoch in range(1, num_epochs + 1):
        metric_name = f"epoch_{epoch}_cost_min"
        run_metric = exp.read_metric(run_path, metric_name)
        # print(run_metric.name)
        epoch_data = pd.DataFrame({
            'epoch': epoch,
            'step': np.arange(1, num_steps + 1),
            'model_path': run_metric.model_path,
            'value': run_metric.data  # assuming this is a list or array of length num_steps
        })
        data_list.append(epoch_data)

    # Concatenate all dataframes
    data = pd.concat(data_list)

    # calculate mean values for line plot
    mean_values = data.groupby('step')['value'].mean()

    # create box plots for each step
    for step in range(1, num_steps+1):
        fig.add_trace(go.Box(
            y=data.loc[data['step'] == step, 'value'],
            name=str(step),
            showlegend=False,
            boxpoints='outliers',
            marker=dict(
              size=0.5,
              color=colors[opt_idx],  # use different color for each optimizer
              line=dict(width=0.3),  # change width of the box outline
            ),
            line=dict(width=0.3),  # change width of the whisker lines
        ))

    # create line plot for the mean values
    fig.add_trace(go.Scatter(
        x=mean_values.index,
        y=mean_values.values,
        mode='lines',
        # name=f'mean optimizer {opt_idx+1}',
        name=name_dict[run_metric.model_path],
        # name=run_metric.model_path,
        line=dict(
          color=colors[opt_idx],
          width=2,  # adjust this for line thickness
        ),
    ))


fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=500,
    height=500,
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(255, 255, 255, 0.8)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=1
    )
)

fig.update_xaxes(
    title_text='Evaluation iterations',
    tickangle=0,
    tickvals=[i for i in range(0,261,50)],
    # range=[0, 150],
    )
fig.update_yaxes(
    title_text='Loss',
    # range=[0, 2.1],
    )


fig.show()

In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=400,
    height=500,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,10)],
#     range=[0, 50],
    )
fig.update_yaxes(
    title_text='Loss',
#     range=[0.2, 1.6],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/ackley_2_line_zoom1.pdf', scale=10) #width=800, height=600, scale=2


In [ ]:
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)',  # white background,
    xaxis=dict(
        gridcolor='rgb(200, 200, 200)',  # light gray gridlines
        zerolinecolor='rgb(200, 200, 200)'  # light gray zero line
    ),
    yaxis=dict(
        gridcolor='rgb(200, 200, 200)',
        zerolinecolor='rgb(200, 200, 200)'
    ),
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    autosize=False,
    width=350,
    height=500,
    showlegend=False,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor='right',
        yanchor='top',
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        # bgcolor="LightSteelBlue",
        bgcolor='rgba(229, 236, 246, 1)',  # LightSteelBlue with 50% transparency
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(
    title_text='Evaluation iteractions',
    tickangle=0,
    tickvals=[i for i in range(0,261,1)],
    range=[246, 250],
    )
fig.update_yaxes(
    title_text='Loss',
    range=[0.125, 0.42],
    )
fig.show()

In [ ]:
import plotly.io as pio

pio.write_image(fig, './figures/ackley_5_line_zoom2.pdf', scale=10) #width=800, height=600, scale=2

# Comparação Geral

In [1]:
import pandas as pd
# Substitua 'caminho_do_seu_arquivo.csv' pelo caminho real do seu arquivo
df = pd.read_csv('./figures/tabelas_latex_csv.csv', encoding='cp1252', sep=';')


In [2]:
df

,Método,Dimensão,Cenário de Treino,Cenário de Teste,Média Fitness Mín.,Desv. Padrão Fitness Mín.
0,referencia,5,mínimos quadrados,mínimos quadrados,0.3136,0.0593
1,proposto,5,mínimos quadrados,mínimos quadrados,0.1508,0.0335
2,referencia,5,rastrigin,mínimos quadrados,0.2876,0.0181
3,proposto,5,rastrigin,mínimos quadrados,0.1603,0.0127
4,referencia,5,rosenbrock,mínimos quadrados,0.3252,0.0192
...,...,...,...,...,...,...
115,proposto,2,weierstrass,ackley,6.7712,0.0025
116,referencia,2,ackley,ackley,0.0014,0.0001
117,proposto,2,ackley,ackley,0.0023,0.0001
118,Adam,2,-,ackley,-,-


In [3]:
# Converta as colunas de string para float
df['Média Fitness Mín.'] = pd.to_numeric(df['Média Fitness Mín.'], errors='coerce')
df['Desv. Padrão Fitness Mín.'] = pd.to_numeric(df['Desv. Padrão Fitness Mín.'], errors='coerce')

In [4]:
df

,Método,Dimensão,Cenário de Treino,Cenário de Teste,Média Fitness Mín.,Desv. Padrão Fitness Mín.
0,referencia,5,mínimos quadrados,mínimos quadrados,0.3136,0.0593
1,proposto,5,mínimos quadrados,mínimos quadrados,0.1508,0.0335
2,referencia,5,rastrigin,mínimos quadrados,0.2876,0.0181
3,proposto,5,rastrigin,mínimos quadrados,0.1603,0.0127
4,referencia,5,rosenbrock,mínimos quadrados,0.3252,0.0192
...,...,...,...,...,...,...
115,proposto,2,weierstrass,ackley,6.7712,0.0025
116,referencia,2,ackley,ackley,0.0014,0.0001
117,proposto,2,ackley,ackley,0.0023,0.0001
118,Adam,2,-,ackley,NaN,NaN


In [5]:
# DataFrame onde "Cenário de Treino" é igual a "Cenário de Teste"
# df_iguais = df[df['Cenário de Treino'] == df['Cenário de Teste']]
mask = (df['Cenário de Treino'] == df['Cenário de Teste']) | (df['Método'].isin(['PSO', 'Adam']))
df_iguais = df[mask]

# DataFrame onde "Cenário de Treino" é diferente de "Cenário de Teste"
df_diferentes = df[df['Cenário de Treino'] != df['Cenário de Teste']]


In [6]:
df_iguais


,Método,Dimensão,Cenário de Treino,Cenário de Teste,Média Fitness Mín.,Desv. Padrão Fitness Mín.
0,referencia,5,mínimos quadrados,mínimos quadrados,0.3136,0.0593
1,proposto,5,mínimos quadrados,mínimos quadrados,0.1508,0.0335
10,Adam,5,-,mínimos quadrados,0.3814,0.0070
11,PSO,5,-,mínimos quadrados,0.1531,0.1301
12,referencia,2,mínimos quadrados,mínimos quadrados,0.2350,0.0181
13,proposto,2,mínimos quadrados,mínimos quadrados,0.2435,0.0185
22,Adam,2,-,mínimos quadrados,0.2968,0.0028
23,PSO,2,-,mínimos quadrados,0.0022,0.0045
26,referencia,5,rastrigin,rastrigin,8.1353,0.2936
27,proposto,5,rastrigin,rastrigin,8.0687,0.2767


In [7]:
# Função para normalizar dentro de cada grupo de 'Cenário de Treino' e 'Dimensão'
def normalize(group):
    group['Pontuação'] = (group['Média Fitness Mín.'] - group['Média Fitness Mín.'].max()).abs() / (group['Média Fitness Mín.'].max() - 0)
    return group

# df_iguais = df_iguais[~df_iguais.apply(lambda row: row.astype(str).str.contains('weierstrass').any(), axis=1)]
df_iguais = df_iguais.groupby(['Cenário de Teste', 'Dimensão']).apply(normalize)

# Remova as linhas com NaN, se desejar
df_iguais = df_iguais.dropna()

# df_iguais = df_iguais.drop(columns=['Média Fitness Mín.', 'Desv. Padrão Fitness Mín.'])


In [8]:
df_iguais

,Método,Dimensão,Cenário de Treino,Cenário de Teste,Média Fitness Mín.,Desv. Padrão Fitness Mín.,Pontuação
0,referencia,5,mínimos quadrados,mínimos quadrados,0.3136,0.0593,0.177766
1,proposto,5,mínimos quadrados,mínimos quadrados,0.1508,0.0335,0.604615
10,Adam,5,-,mínimos quadrados,0.3814,0.0070,0.000000
11,PSO,5,-,mínimos quadrados,0.1531,0.1301,0.598584
12,referencia,2,mínimos quadrados,mínimos quadrados,0.2350,0.0181,0.208221
13,proposto,2,mínimos quadrados,mínimos quadrados,0.2435,0.0185,0.179582
22,Adam,2,-,mínimos quadrados,0.2968,0.0028,0.000000
23,PSO,2,-,mínimos quadrados,0.0022,0.0045,0.992588
26,referencia,5,rastrigin,rastrigin,8.1353,0.2936,0.722072
27,proposto,5,rastrigin,rastrigin,8.0687,0.2767,0.724347


In [9]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


In [10]:
df_iguais.head()

,Método,Dimensão,Cenário de Treino,Cenário de Teste,Média Fitness Mín.,Desv. Padrão Fitness Mín.,Pontuação
0,referencia,5,mínimos quadrados,mínimos quadrados,0.3136,0.0593,0.177766
1,proposto,5,mínimos quadrados,mínimos quadrados,0.1508,0.0335,0.604615
10,Adam,5,-,mínimos quadrados,0.3814,0.0070,0.000000
11,PSO,5,-,mínimos quadrados,0.1531,0.1301,0.598584
12,referencia,2,mínimos quadrados,mínimos quadrados,0.2350,0.0181,0.208221


In [11]:
df_iguais_agg = df_iguais.pivot_table(index='Método', values='Pontuação', aggfunc='mean').reset_index()
df_iguais_agg

,Método,Pontuação
0,Adam,0.000000
1,PSO,0.728095
2,proposto,0.840940
3,referencia,0.777992


In [12]:
df_iguais_dim = df_iguais.pivot_table(index=['Método','Dimensão'], values='Pontuação', aggfunc='mean').reset_index()

In [13]:
def color_scale(value):
    red = 1 - value
    green = value
    return f"\\cellcolor[rgb]{{{red:.4f},{green:.4f},0.0000}}{value:.6f}"

In [14]:
# df_iguais_dim = df_iguais_dim.groupby(['Método','Dimensão'])["Pontuação"].mean().reset_index()

In [15]:
df_iguais_dim2 = df_iguais_dim[df_iguais_dim['Dimensão']==2]
df_iguais_dim2

,Método,Dimensão,Pontuação
0,Adam,2,0.000000
2,PSO,2,0.785773
4,proposto,2,0.817390
6,referencia,2,0.813713


In [16]:
df_iguais_dim2 = df_iguais_dim2.copy()
df_iguais_dim2["Colored Pontuação"] = df_iguais_dim2["Pontuação"].apply(color_scale)

In [17]:
df_iguais_dim2_latex_table = df_iguais_dim2.to_latex(index=False, columns=["Método","Dimensão","Colored Pontuação"], escape=False, header=["Método","Dimensão", "Pontuação"])
print(df_iguais_dim2_latex_table)

\begin{tabular}{lrl}
\toprule
    Método & Dimensão &                                     Pontuação \\
\midrule
      Adam &        2 & \cellcolor[rgb]{1.0000,0.0000,0.0000}0.000000 \\
       PSO &        2 & \cellcolor[rgb]{0.2142,0.7858,0.0000}0.785773 \\
  proposto &        2 & \cellcolor[rgb]{0.1826,0.8174,0.0000}0.817390 \\
referencia &        2 & \cellcolor[rgb]{0.1863,0.8137,0.0000}0.813713 \\
\bottomrule
\end{tabular}



In [18]:
df_iguais_dim5 = df_iguais_dim[df_iguais_dim['Dimensão']==5]
df_iguais_dim5

,Método,Dimensão,Pontuação
1,Adam,5,0.000000
3,PSO,5,0.670416
5,proposto,5,0.864490
7,referencia,5,0.742272


In [19]:
df_iguais_dim5 = df_iguais_dim5.copy()
df_iguais_dim5["Colored Pontuação"] = df_iguais_dim5["Pontuação"].apply(color_scale)
df_iguais_dim5_latex_table = df_iguais_dim5.to_latex(index=False, columns=["Método","Dimensão","Colored Pontuação"], escape=False, header=["Método","Dimensão", "Pontuação"])
print(df_iguais_dim5_latex_table)

\begin{tabular}{lrl}
\toprule
    Método & Dimensão &                                     Pontuação \\
\midrule
      Adam &        5 & \cellcolor[rgb]{1.0000,0.0000,0.0000}0.000000 \\
       PSO &        5 & \cellcolor[rgb]{0.3296,0.6704,0.0000}0.670416 \\
  proposto &        5 & \cellcolor[rgb]{0.1355,0.8645,0.0000}0.864490 \\
referencia &        5 & \cellcolor[rgb]{0.2577,0.7423,0.0000}0.742272 \\
\bottomrule
\end{tabular}



In [20]:
df_iguais_agg["Colored Pontuação"] = df_iguais_agg["Pontuação"].apply(color_scale)
df_iguais_agg_latex_table = df_iguais_agg.to_latex(index=False, columns=["Método", "Colored Pontuação"], escape=False, header=["Método", "Pontuação"])

In [21]:
print(df_iguais_agg_latex_table)

\begin{tabular}{ll}
\toprule
    Método &                                     Pontuação \\
\midrule
      Adam & \cellcolor[rgb]{1.0000,0.0000,0.0000}0.000000 \\
       PSO & \cellcolor[rgb]{0.2719,0.7281,0.0000}0.728095 \\
  proposto & \cellcolor[rgb]{0.1591,0.8409,0.0000}0.840940 \\
referencia & \cellcolor[rgb]{0.2220,0.7780,0.0000}0.777992 \\
\bottomrule
\end{tabular}



In [22]:
# Suponho que seu DataFrame seja chamado df
# df_encoded = pd.get_dummies(df_iguais, columns=['Método', 'Cenário de Treino','Cenário de Teste'])
# df_encoded = pd.get_dummies(pivot_df, columns=['Método'])
# df_encoded = df_encoded.drop(columns=['Média Fitness Mín.', 'Desv. Padrão Fitness Mín.'])

In [23]:
# # Criar figura e eixos
# plt.figure(figsize=(12, 8))

# # Plotar heatmap
# sns.heatmap(df_encoded.corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)

# # Configurar título e mostrar gráfico
# plt.title('Correlação entre Métodos e Média Fitness Mín. Normalizada')
# plt.show()


In [24]:
df_diferentes

,Método,Dimensão,Cenário de Treino,Cenário de Teste,Média Fitness Mín.,Desv. Padrão Fitness Mín.
2,referencia,5,rastrigin,mínimos quadrados,0.2876,0.0181
3,proposto,5,rastrigin,mínimos quadrados,0.1603,0.0127
4,referencia,5,rosenbrock,mínimos quadrados,0.3252,0.0192
5,proposto,5,rosenbrock,mínimos quadrados,0.3677,0.0228
6,referencia,5,weierstrass,mínimos quadrados,314.5579,7.4786
...,...,...,...,...,...,...
113,proposto,2,rosenbrock,ackley,0.0072,0.0009
114,referencia,2,weierstrass,ackley,9.2975,0.0028
115,proposto,2,weierstrass,ackley,6.7712,0.0025
118,Adam,2,-,ackley,NaN,NaN


In [25]:
# Função para normalizar dentro de cada grupo de 'Cenário de Treino' e 'Dimensão'
def normalize(group):
    group['Pontuação'] = (group['Média Fitness Mín.'] - group['Média Fitness Mín.'].max()).abs() / (group['Média Fitness Mín.'].max() - 0)
    return group

# df_diferentes = df_diferentes[~df_diferentes.apply(lambda row: row.astype(str).str.contains('weierstrass').any(), axis=1)]
df_diferentes = df_diferentes.groupby(['Cenário de Teste', 'Dimensão']).apply(normalize)

# Remova as linhas com NaN, se desejar
df_diferentes = df_diferentes.dropna()

# df_diferentes = df_diferentes.drop(columns=['Média Fitness Mín.', 'Desv. Padrão Fitness Mín.'])

In [26]:
df_diferentes_agg = df_diferentes.pivot_table(index='Método', values='Pontuação', aggfunc='mean').reset_index()

In [27]:
df_diferentes_agg

,Método,Pontuação
0,Adam,0.667139
1,PSO,0.982569
2,proposto,0.938230
3,referencia,0.792445


In [28]:
df_diferentes_agg["Colored Pontuação"] = df_diferentes_agg["Pontuação"].apply(color_scale)
df_diferentes_agg_latex_table = df_diferentes_agg.to_latex(index=False, columns=["Método", "Colored Pontuação"], escape=False, header=["Método", "Pontuação"])
print(df_diferentes_agg_latex_table)

\begin{tabular}{ll}
\toprule
    Método &                                     Pontuação \\
\midrule
      Adam & \cellcolor[rgb]{0.3329,0.6671,0.0000}0.667139 \\
       PSO & \cellcolor[rgb]{0.0174,0.9826,0.0000}0.982569 \\
  proposto & \cellcolor[rgb]{0.0618,0.9382,0.0000}0.938230 \\
referencia & \cellcolor[rgb]{0.2076,0.7924,0.0000}0.792445 \\
\bottomrule
\end{tabular}



In [29]:
df_diferentes_dim = df_diferentes.pivot_table(index=['Método','Dimensão'], values='Pontuação', aggfunc='mean').reset_index()

In [30]:
df_diferentes_dim2 = df_diferentes_dim[df_diferentes_dim['Dimensão']==2]
df_diferentes_dim2

,Método,Dimensão,Pontuação
0,Adam,2,0.651297
2,PSO,2,0.993220
4,proposto,2,0.902687
6,referencia,2,0.799418


In [31]:
df_diferentes_dim2 = df_diferentes_dim2.copy()
df_diferentes_dim2["Colored Pontuação"] = df_diferentes_dim2["Pontuação"].apply(color_scale)

In [32]:
df_diferentes_dim2_latex_table = df_diferentes_dim2.to_latex(index=False, columns=["Método","Dimensão","Colored Pontuação"], escape=False, header=["Método","Dimensão", "Pontuação"])
print(df_diferentes_dim2_latex_table)

\begin{tabular}{lrl}
\toprule
    Método & Dimensão &                                     Pontuação \\
\midrule
      Adam &        2 & \cellcolor[rgb]{0.3487,0.6513,0.0000}0.651297 \\
       PSO &        2 & \cellcolor[rgb]{0.0068,0.9932,0.0000}0.993220 \\
  proposto &        2 & \cellcolor[rgb]{0.0973,0.9027,0.0000}0.902687 \\
referencia &        2 & \cellcolor[rgb]{0.2006,0.7994,0.0000}0.799418 \\
\bottomrule
\end{tabular}



In [33]:
df_diferentes_dim5 = df_diferentes_dim[df_iguais_dim['Dimensão']==5]
df_diferentes_dim5

,Método,Dimensão,Pontuação
1,Adam,5,0.682982
3,PSO,5,0.971918
5,proposto,5,0.973774
7,referencia,5,0.785472


In [34]:
df_diferentes_dim5 = df_diferentes_dim5.copy()
df_diferentes_dim5["Colored Pontuação"] = df_diferentes_dim5["Pontuação"].apply(color_scale)

In [35]:
df_diferentes_dim5_latex_table = df_diferentes_dim5.to_latex(index=False, columns=["Método","Dimensão","Colored Pontuação"], escape=False, header=["Método","Dimensão", "Pontuação"])
print(df_diferentes_dim5_latex_table)

\begin{tabular}{lrl}
\toprule
    Método & Dimensão &                                     Pontuação \\
\midrule
      Adam &        5 & \cellcolor[rgb]{0.3170,0.6830,0.0000}0.682982 \\
       PSO &        5 & \cellcolor[rgb]{0.0281,0.9719,0.0000}0.971918 \\
  proposto &        5 & \cellcolor[rgb]{0.0262,0.9738,0.0000}0.973774 \\
referencia &        5 & \cellcolor[rgb]{0.2145,0.7855,0.0000}0.785472 \\
\bottomrule
\end{tabular}



## por função

In [36]:
df_diferentes

,Método,Dimensão,Cenário de Treino,Cenário de Teste,Média Fitness Mín.,Desv. Padrão Fitness Mín.,Pontuação
2,referencia,5,rastrigin,mínimos quadrados,0.2876,0.0181,0.999086
3,proposto,5,rastrigin,mínimos quadrados,0.1603,0.0127,0.999490
4,referencia,5,rosenbrock,mínimos quadrados,0.3252,0.0192,0.998966
5,proposto,5,rosenbrock,mínimos quadrados,0.3677,0.0228,0.998831
6,referencia,5,weierstrass,mínimos quadrados,314.5579,7.4786,0.000000
...,...,...,...,...,...,...,...
112,referencia,2,rosenbrock,ackley,0.0334,0.0018,0.996408
113,proposto,2,rosenbrock,ackley,0.0072,0.0009,0.999226
114,referencia,2,weierstrass,ackley,9.2975,0.0028,0.000000
115,proposto,2,weierstrass,ackley,6.7712,0.0025,0.271718


In [37]:
df_diferentes_prob = df_diferentes.pivot_table(index=['Cenário de Treino'], values='Pontuação', aggfunc='mean').reset_index()
df_diferentes_prob

,Cenário de Treino,Pontuação
0,-,0.842378
1,ackley,0.986702
2,mínimos quadrados,0.979286
3,rastrigin,0.997395
4,rosenbrock,0.977239
5,weierstrass,0.386065


In [38]:
df_diferentes_prob_dim = df_diferentes.pivot_table(index=['Cenário de Treino', 'Dimensão'], values='Pontuação', aggfunc='mean').reset_index()
df_diferentes_prob_dim

,Cenário de Treino,Dimensão,Pontuação
0,-,2,0.841254
1,-,5,0.843502
2,ackley,2,0.993601
3,ackley,5,0.979803
4,mínimos quadrados,2,0.988468
5,mínimos quadrados,5,0.970105
6,rastrigin,2,0.996031
7,rastrigin,5,0.998758
8,rosenbrock,2,0.995065
9,rosenbrock,5,0.959413


In [39]:
# df_diferentes_prob_dim = df_diferentes_prob_dim.copy()
df_diferentes_prob_dim2 = df_diferentes_prob_dim[df_diferentes_prob_dim['Dimensão']==2]
df_diferentes_prob_dim2

,Cenário de Treino,Dimensão,Pontuação
0,-,2,0.841254
2,ackley,2,0.993601
4,mínimos quadrados,2,0.988468
6,rastrigin,2,0.996031
8,rosenbrock,2,0.995065
10,weierstrass,2,0.282096


In [40]:
df_diferentes_prob_dim5 = df_diferentes_prob_dim[df_diferentes_prob_dim['Dimensão']==5]
df_diferentes_prob_dim5

,Cenário de Treino,Dimensão,Pontuação
1,-,5,0.843502
3,ackley,5,0.979803
5,mínimos quadrados,5,0.970105
7,rastrigin,5,0.998758
9,rosenbrock,5,0.959413
11,weierstrass,5,0.490034


# Para revisão

In [65]:
import pandas as pd
import numpy as np

# Substitua 'caminho_do_seu_arquivo.csv' pelo caminho real do seu arquivo
df = pd.read_csv('./figures/tabelas_latex_csv.csv', encoding='cp1252', sep=';')

In [66]:
# Converta as colunas de string para float
df['Média Fitness Mín.'] = pd.to_numeric(df['Média Fitness Mín.'], errors='coerce')
df['Desv. Padrão Fitness Mín.'] = pd.to_numeric(df['Desv. Padrão Fitness Mín.'], errors='coerce')

df.head()

,Método,Dimensão,Cenário de Treino,Cenário de Teste,Média Fitness Mín.,Desv. Padrão Fitness Mín.
0,referencia,5,mínimos quadrados,mínimos quadrados,0.3136,0.0593
1,proposto,5,mínimos quadrados,mínimos quadrados,0.1508,0.0335
2,referencia,5,rastrigin,mínimos quadrados,0.2876,0.0181
3,proposto,5,rastrigin,mínimos quadrados,0.1603,0.0127
4,referencia,5,rosenbrock,mínimos quadrados,0.3252,0.0192


In [67]:
# Função para normalizar dentro de cada grupo de 'Cenário de Treino' e 'Dimensão'
def normalize(group):
    group['Pontuação'] = (group['Média Fitness Mín.'] - group['Média Fitness Mín.'].max()).abs() / (group['Média Fitness Mín.'].max() - 0)
    return group

# df_iguais = df_iguais[~df_iguais.apply(lambda row: row.astype(str).str.contains('weierstrass').any(), axis=1)]
df = df.groupby(['Cenário de Teste', 'Dimensão']).apply(normalize)

# Remova as linhas com NaN, se desejar
df = df.dropna()

# df_iguais = df_iguais.drop(columns=['Média Fitness Mín.', 'Desv. Padrão Fitness Mín.'])

In [68]:
df['Cenários Treino e Teste Iguais'] = np.where(df['Cenário de Treino'] == df['Cenário de Teste'], 'sim', 'não')

In [69]:
df

,Método,Dimensão,Cenário de Treino,Cenário de Teste,Média Fitness Mín.,Desv. Padrão Fitness Mín.,Pontuação,Cenários Treino e Teste Iguais
0,referencia,5,mínimos quadrados,mínimos quadrados,0.3136,0.0593,0.999003,sim
1,proposto,5,mínimos quadrados,mínimos quadrados,0.1508,0.0335,0.999521,sim
2,referencia,5,rastrigin,mínimos quadrados,0.2876,0.0181,0.999086,não
3,proposto,5,rastrigin,mínimos quadrados,0.1603,0.0127,0.999490,não
4,referencia,5,rosenbrock,mínimos quadrados,0.3252,0.0192,0.998966,não
...,...,...,...,...,...,...,...,...
114,referencia,2,weierstrass,ackley,9.2975,0.0028,0.000000,não
115,proposto,2,weierstrass,ackley,6.7712,0.0025,0.271718,não
116,referencia,2,ackley,ackley,0.0014,0.0001,0.999849,sim
117,proposto,2,ackley,ackley,0.0023,0.0001,0.999753,sim


In [70]:
df_train_test_no = df[df['Cenários Treino e Teste Iguais']=='não']
df_train_test_no = df_train_test_no[df_train_test_no['Cenário de Treino']!='-']

In [71]:
df_train_test_no

,Método,Dimensão,Cenário de Treino,Cenário de Teste,Média Fitness Mín.,Desv. Padrão Fitness Mín.,Pontuação,Cenários Treino e Teste Iguais
2,referencia,5,rastrigin,mínimos quadrados,0.2876,0.0181,0.999086,não
3,proposto,5,rastrigin,mínimos quadrados,0.1603,0.0127,0.999490,não
4,referencia,5,rosenbrock,mínimos quadrados,0.3252,0.0192,0.998966,não
5,proposto,5,rosenbrock,mínimos quadrados,0.3677,0.0228,0.998831,não
6,referencia,5,weierstrass,mínimos quadrados,314.5579,7.4786,0.000000,não
...,...,...,...,...,...,...,...,...
111,proposto,2,rastrigin,ackley,0.0025,0.0004,0.999731,não
112,referencia,2,rosenbrock,ackley,0.0334,0.0018,0.996408,não
113,proposto,2,rosenbrock,ackley,0.0072,0.0009,0.999226,não
114,referencia,2,weierstrass,ackley,9.2975,0.0028,0.000000,não


In [72]:
df_agg = df.pivot_table(index=['Método','Dimensão', 'Cenários Treino e Teste Iguais'], values='Pontuação', aggfunc='mean').reset_index()

In [73]:
df_agg

,Método,Dimensão,Cenários Treino e Teste Iguais,Pontuação
0,Adam,2,não,0.651297
1,Adam,5,não,0.682982
2,PSO,2,não,0.993220
3,PSO,5,não,0.971918
4,proposto,2,não,0.902687
5,proposto,2,sim,0.983415
6,proposto,5,não,0.973774
7,proposto,5,sim,0.984757
8,referencia,2,não,0.799418
9,referencia,2,sim,0.976065


In [74]:
df_agg.loc[df_agg['Método'] == 'Adam', 'Cenários Treino e Teste Iguais'] = '-'

In [75]:
df_agg.loc[df_agg['Método'] == 'PSO', 'Cenários Treino e Teste Iguais'] = '-'

In [76]:
df_agg = df_agg.sort_values(by=['Cenários Treino e Teste Iguais', 'Dimensão'], ascending=False)

In [77]:
df_agg

,Método,Dimensão,Cenários Treino e Teste Iguais,Pontuação
7,proposto,5,sim,0.984757
11,referencia,5,sim,0.968277
5,proposto,2,sim,0.983415
9,referencia,2,sim,0.976065
6,proposto,5,não,0.973774
10,referencia,5,não,0.785472
4,proposto,2,não,0.902687
8,referencia,2,não,0.799418
1,Adam,5,-,0.682982
3,PSO,5,-,0.971918


In [78]:
df_agg["Colored Pontuação"] = df_agg["Pontuação"].apply(color_scale)

In [79]:
# Assuming df_agg is your DataFrame
# Sort the DataFrame if needed (this is just an example, sort by your desired column)
# df_agg = df_agg.sort_values(by='YourSortColumn', ascending=False)

# Convert to LaTeX
df_agg_latex = df_agg.to_latex(index=False, 
                               columns=["Método", "Dimensão", "Cenários Treino e Teste Iguais", "Colored Pontuação"], 
                               escape=False, 
                               header=["Método", "Dimensão", "Cenários Treino e Teste Iguais", "Colored Pontuação"])

print(df_agg_latex)


\begin{tabular}{lrll}
\toprule
    Método & Dimensão & Cenários Treino e Teste Iguais &                             Colored Pontuação \\
\midrule
  proposto &        5 &                            sim & \cellcolor[rgb]{0.0152,0.9848,0.0000}0.984757 \\
referencia &        5 &                            sim & \cellcolor[rgb]{0.0317,0.9683,0.0000}0.968277 \\
  proposto &        2 &                            sim & \cellcolor[rgb]{0.0166,0.9834,0.0000}0.983415 \\
referencia &        2 &                            sim & \cellcolor[rgb]{0.0239,0.9761,0.0000}0.976065 \\
  proposto &        5 &                            não & \cellcolor[rgb]{0.0262,0.9738,0.0000}0.973774 \\
referencia &        5 &                            não & \cellcolor[rgb]{0.2145,0.7855,0.0000}0.785472 \\
  proposto &        2 &                            não & \cellcolor[rgb]{0.0973,0.9027,0.0000}0.902687 \\
referencia &        2 &                            não & \cellcolor[rgb]{0.2006,0.7994,0.0000}0.799418 \\
      

# Per Function

In [80]:
df_train_test_no_agg = df_train_test_no.pivot_table(index=['Dimensão', 'Cenário de Treino'], values='Pontuação', aggfunc='mean').reset_index()

In [81]:
df_train_test_no_agg = df_train_test_no_agg.sort_values(by=['Dimensão','Pontuação'], ascending=False)

In [82]:
df_train_test_no_agg

,Dimensão,Cenário de Treino,Pontuação
7,5,rastrigin,0.998758
5,5,ackley,0.979803
6,5,mínimos quadrados,0.970105
8,5,rosenbrock,0.959413
9,5,weierstrass,0.490034
2,2,rastrigin,0.996031
3,2,rosenbrock,0.995065
0,2,ackley,0.993601
1,2,mínimos quadrados,0.988468
4,2,weierstrass,0.282096


In [83]:
df_train_test_no_agg2 = df_train_test_no.pivot_table(index=['Método','Dimensão', 'Cenário de Treino'], values='Pontuação', aggfunc='mean').reset_index()

In [84]:
df_train_test_no_agg2 = df_train_test_no_agg2.sort_values(by=['Dimensão','Pontuação'], ascending=False)
df_train_test_no_agg2[df_train_test_no_agg2['Método']=='proposto']

,Método,Dimensão,Cenário de Treino,Pontuação
7,proposto,5,rastrigin,0.998832
9,proposto,5,weierstrass,0.980068
5,proposto,5,ackley,0.978818
6,proposto,5,mínimos quadrados,0.972289
8,proposto,5,rosenbrock,0.938860
1,proposto,2,mínimos quadrados,0.996854
2,proposto,2,rastrigin,0.995328
3,proposto,2,rosenbrock,0.994489
0,proposto,2,ackley,0.992992
4,proposto,2,weierstrass,0.533774


In [85]:
df_train_test_no_agg2[df_train_test_no_agg2['Método']=='referencia']

,Método,Dimensão,Cenário de Treino,Pontuação
17,referencia,5,rastrigin,0.998685
15,referencia,5,ackley,0.980788
18,referencia,5,rosenbrock,0.979965
16,referencia,5,mínimos quadrados,0.967920
19,referencia,5,weierstrass,0.000000
12,referencia,2,rastrigin,0.996735
13,referencia,2,rosenbrock,0.995641
10,referencia,2,ackley,0.994211
11,referencia,2,mínimos quadrados,0.980082
14,referencia,2,weierstrass,0.030418


In [86]:
df_train_test_no_agg2

,Método,Dimensão,Cenário de Treino,Pontuação
7,proposto,5,rastrigin,0.998832
17,referencia,5,rastrigin,0.998685
15,referencia,5,ackley,0.980788
9,proposto,5,weierstrass,0.980068
18,referencia,5,rosenbrock,0.979965
5,proposto,5,ackley,0.978818
6,proposto,5,mínimos quadrados,0.972289
16,referencia,5,mínimos quadrados,0.967920
8,proposto,5,rosenbrock,0.938860
19,referencia,5,weierstrass,0.000000


In [87]:
df_train_test_no_agg2["Colored Pontuação"] = df_train_test_no_agg2["Pontuação"].apply(color_scale)
df_train_test_no_agg_ = df_train_test_no_agg2.copy()
df_train_test_no_agg_["Colored Pontuação"] = df_train_test_no_agg_["Pontuação"].apply(color_scale)

In [89]:
# Convert to LaTeX
df_train_test_no_agg2_latex = df_train_test_no_agg_.to_latex(index=False, 
                               columns=["Método", "Dimensão", "Cenário de Treino", "Colored Pontuação"], 
                               escape=False, 
                               header=["Método", "Dimensão", "Cenário de Treino", "Colored Pontuação"])

print(df_train_test_no_agg2_latex)


\begin{tabular}{lrll}
\toprule
    Método & Dimensão & Cenário de Treino &                             Colored Pontuação \\
\midrule
  proposto &        5 &         rastrigin & \cellcolor[rgb]{0.0012,0.9988,0.0000}0.998832 \\
referencia &        5 &         rastrigin & \cellcolor[rgb]{0.0013,0.9987,0.0000}0.998685 \\
referencia &        5 &            ackley & \cellcolor[rgb]{0.0192,0.9808,0.0000}0.980788 \\
  proposto &        5 &       weierstrass & \cellcolor[rgb]{0.0199,0.9801,0.0000}0.980068 \\
referencia &        5 &        rosenbrock & \cellcolor[rgb]{0.0200,0.9800,0.0000}0.979965 \\
  proposto &        5 &            ackley & \cellcolor[rgb]{0.0212,0.9788,0.0000}0.978818 \\
  proposto &        5 & mínimos quadrados & \cellcolor[rgb]{0.0277,0.9723,0.0000}0.972289 \\
referencia &        5 & mínimos quadrados & \cellcolor[rgb]{0.0321,0.9679,0.0000}0.967920 \\
  proposto &        5 &        rosenbrock & \cellcolor[rgb]{0.0611,0.9389,0.0000}0.938860 \\
referencia &        5 &       

In [ ]:
df_agg_latex